In [ ]:
!pip install sumy
!pip install bert-extractive-summarizer

!pip install torch transformers nltk
!pip install torch==1.8.2 torchvision==0.9.2 torchaudio===0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html

!pip install transformers
!pip install bert_score --quiet


Looking in links: https://download.pytorch.org/whl/lts/1.8/torch_lts.html
ERROR: Could not find a version that satisfies the requirement torch==1.8.2 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2)
ERROR: No matching distribution found for torch==1.8.2


In [ ]:
pip install rouge

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import nltk
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from summarizer import Summarizer
from transformers import BartTokenizer, BartForConditionalGeneration, T5Tokenizer, T5ForConditionalGeneration, BertModel, BertTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity
import heapq
import pandas as pd
import numpy as np
import re
from nltk.translate import meteor_score
from bert_score import BERTScorer
from rouge import Rouge
from nltk.tokenize import word_tokenize
from transformers import PegasusTokenizer, PegasusForConditionalGeneration


In [ ]:
from nltk.tokenize import word_tokenize
from nltk.translate import meteor_score
from bert_score import BERTScorer
from rouge import Rouge

def evaluate_summaries(generated_summary, reference_summary):
    meteor_scores = {}
    bert_scores = {}
    rouge_scores = {}

    # Tokenize reference summary
    reference_summary_tokens = word_tokenize(reference_summary.lower())

    # Tokenize generated summary
    generated_summary_tokens = word_tokenize(generated_summary.lower())

    # Initialize Rouge
    rouge = Rouge()

    # Calculate METEOR score
    meteor_score_value = meteor_score.meteor_score([reference_summary_tokens], generated_summary_tokens)
    meteor_scores['Generated'] = meteor_score_value

    # Calculate BERTScore
    scorer = BERTScorer(lang="en")
    precision, recall, bert_score_value = scorer.score([reference_summary], [generated_summary])
    bert_scores['Generated'] = bert_score_value.item()

    # Calculate ROUGE scores
    rouge_score = rouge.get_scores(generated_summary, reference_summary, avg=True)
    rouge_scores['Generated'] = rouge_score

    return meteor_scores, bert_scores, rouge_scores



SUMMARIES

<b>Comedy of errors</b>  reference_summary




In [ ]:
reference_summary="""Duke Solinus, ruler of Ephesus, presides over the trial of Egeon of Syracuse. Syracusians are not allowed in Ephesus, so Egeon has been detained. When asked why he is now in Ephesus, Egeon explains how he is searching for his lost twin sons and their twin servants.

Twenty three years before, Egeon had lost his wife and one of their identical twin sons (along with the boy's slave) in a storm at sea. Egeon had brought up the surviving boy and his slave. At eighteen, they had gone in search of their lost brothers. After no word from them, Egeon had also left home to seek news in Ephesus. Solinus is softened by the story and allows Egeon until sunset to try to raise 1,000 marks as a ransom or else he must die.
Meanwhile, in a nearby marketplace, a merchant has befriended two tourists: Antipholus of Syracuse (Egeon's son) and his servant companion, Dromio. Learning of the ban on Syracusians, they put on local dress before going to explore the town. Unknown to them, their twin brothers (with identical names) have been living there after being saved from the storm by fishermen.
Antipholus of Syracuse is very surprised to be accosted by Dromio of Ephesus. Dromio is angry that his master has not returned home to his wife, Adriana, for dinner. The likeness of the Dromio twins, and also the sons of Egeon, leads to a series of confusions. This is especially true when Antipholus of Syracuse dines with his sister-in-law and falls in love with her sister, Luciana. His servant, Dromio of Syracuse, refuses to open the door to anyone. He bars the door even when Antipholus of Ephesus returns home with his merchant friends. This exclusion enrages Antipholus of Ephesus and leads him to dine with his friend, the courtesan
A gold chain that Antipholus of Ephesus has ordered is delivered to Antipholus of Syracuse instead. The goldsmith’s claim for payment leads to the arrest of Antipholus of Ephesus and his servant. They refuse to pay for a chain that they did not receive. Adriana, fearing for her husband's sanity, gets the schoolmaster to exorcise him and Dromio (both of Ephesus). While they are under restraint, their Syracusian brothers cause panic in the town. The people think that the Ephesian brothers have somehow escaped. The Syracusian brothers, also frightened, take refuge in a priory or abbey.
The sunset hour of Egeon's sentence is soon approaching. The Duke returns, but is stopped by Adriana, who appeals for aid for her husband. The Ephesian twins escape their bonds and arrive to claim justice. Egeon recognises them, or so he thinks, as the boys he brought up in Syracuse. Solinus, the Duke, sends for the Abbess, who appears with the second pair of twins. She further amazes everyone by recognising Egeon and revealing herself as Emilia, his long-lost wife. She had entered a religious order after surviving the storm and fearing that all her family had died.
When all have told their stories, Antipholus of Syracuse renews his attempts to woo his sister-in-law, Luciana. The Duke pardons Egeon, and everyone goes to celebrate with Emilia at the temple. The two Dromios joyfully leave the stage hand in hand."""

EVALUATE METRICS

FIND METRICS

<b>COMEDY OF ERRORS</b> METRICS

In [ ]:
#1
#User query
generated_summary="""thou hast stolen both mine office and my name:
	The one ne'er got me credit, the other mickle blame.
Your cake here is warm within; you stand here in the cold:
	It would make a man mad as a buck to be so bought and sold.
I mean not cuckold-mad; but, sure, he is stark mad.
For a fish without a fin, there's a fowl without a feather:
	If a crow help us in, sirrah, we'll pluck a crow together.
You are sad, Signior Balthazar: pray God, our cheer
	May answer my good will and your good welcome here.
If thou hadst been Dromio to-day in my place,
	Thou wouldst have chang'd thy face for a name, or thy name for an ass.
If Time be in debt and theft, and a sergeant in the way,
	Hath he not reason to turn back an hour in a day?
Yet once again proclaim it publicly,
	If any friend will pay the sum for him,
	He shall not die; so much we tender him.
Nay, forward, old man; do not break off so;
	For we may pity, though not pardon thee.
If I dream not, thou art Æmilia:
	If thou art she, tell me where is that son
	That floated with thee on the fatal raft?
Thee will I love and with thee lead my life:
	Thou hast no husband yet nor I no wife.
Nay, he's a thief too: have you not heard men say,
	That Time comes stealing on by night and day?
A very reverent body; aye, such a one as a man may not speak of, without he say, 'Sir-reverence.'
O, Signior Balthazar, either at flesh or fish,
	A table-full of welcome makes scarce one dainty dish.
Mistress, upon my life, I tell you true;
	I have not breath'd almost, since I did see it.
I pray you, sir, receive the money now,
	For fear you ne'er see chain nor money more.
Thou art Dromio, thou art my man, thou art thyself.
Ne'er may I look on day, nor sleep on night,
	But she tells to your highness simple truth!
I think thou art, in mind, and so am I.



	Nay, master, both in mind and in my shape.
Thus I mend it: Time himself is bald, and therefore to the world's end will have bald followers.
No, no, the bell: 'tis time that I were gone:
	It was two ere I left him, and now the clock strikes one.
Well, sir, learn to jest in good time: there's a time for all things.
But, sirrah, you shall buy this sport as dear
	As all the metal in your shop will answer.
I tell thee, Syracusian, twenty years
	Have I been patron to Antipholus,
	During which time he ne'er saw Syracusa.
I know not at whose suit he is arrested well;
	But he's in a suit of buff which 'rested him, that can I tell.
By computation, and mine host's report,
	I could not speak with Dromio since at first
	I sent him from the mart.
I will not stay to-night for all the town;
	Therefore away, to get our stuff aboard.
If every one knows us and we know none,
	'Tis time, I think, to trudge, pack, and be gone.
My wife, not meanly proud of two such boys,
	Made daily motions for our home return:
	Unwilling I agreed; alas!
Marry, sir, by a rule as plain as the plain bald pate of Father Time himself.
Well met, well met, Master Antipholus.
Belike you thought our love would last too long,
	If it were chain'd together, and therefore came not.
Was there ever any man thus beaten out of season,
	When, in the why and the wherefore is neither rime nor reason?
Not on a band, but on a stronger thing;
	A chain, a chain.
Let him walk from whence he came, lest he catch cold on's feet.
Come, gaoler, bring me where the goldsmith is:
	I long to know the truth hereof at large.
Sir, sir, I shall have law in Ephesus,
	To your notorious shame, I doubt it not.
What I told you then,
	I hope I shall have leisure to make good,
	If this be not a dream I see and hear.
I thought to have ta'en you at the Porpentine;
	The chain unfinish'd made me stay thus long.
Dost thou conjure for wenches, that thou call'st for such store,
	When one is one too many?
soft, sir; hold you still:
	I'll fetch my sister, to get her good will.
I cannot, nor I will not hold me still:
	My tongue, though not my heart, shall have his will.
I am invited, sir, to certain merchants,
	Of whom I hope to make much benefit;
	I crave your pardon.
Go, Dromio: there's the money, bear it straight,
	And bring thy master home immediately.
You spurn me hence, and he will spurn me hither:
	If I last in this service, you must case me in leather.
Dromio, thou drone, thou snail, thou slug, thou sot!
heart and good will you might;
	But surely, master, not a rag of money.
In Syracusa was I born, and wed
	Unto a woman, happy but for me,
	And by me too, had not our hap been bad.
Against my soul's pure truth why labour you
	To make it wander in an unknown field?
I sent you money, sir, to be your bail,
	By Dromio; but I think he brought it not.
O husband, God doth know you din'd at home;
	Where would you had remain'd until this time.
Mistress, both man and master is possess'd:
	I know it by their pale and deadly looks.
There's none but witches do inhabit here,
	And therefore 'tis high time that I were hence.
And true he swore, though yet forsworn he were.
Why, mistress, sure my master is horn-mad.
Faith, I saw not; but I felt it hot in her breath.
Here is neither cheer, sir, nor welcome: we would fain have either.
'tis pity that thou liv'st
	To walk where any honest men resort.
Get thee gone;
	Buy thou a rope, and bring it home to me.
Mistress, respice finem, respect your end; or rather, to prophesy like the parrot, 'Beware the rope's end.'
No longer from head to foot than from hip to hip: she is spherical, like a globe; I could find out countries in her.
Thyself I call it, being strange to me,
	That, undividable, incorporate,
	Am better than thy dear self's better part.
My lord, in truth, thus far I witness with him,
	That he din'd not at home, but was lock'd out.
I buy a thousand pound a year: I buy a rope!
Many a man would take you at your word,
	And go indeed, having so good a mean.
And so do I; yet did she call me so;
	And this fair gentlewoman, her sister here,
	Did call me brother.
Most mighty duke, behold a man much wrong'd.
Therefore, most gracious duke, with thy command
	Let him be brought forth, and borne hence for help.
Either send the chain or send by me some token.
You sent me for a rope's end as soon:
	You sent me to the bay, sir, for a bark.
That very hour, and in the self-same inn,
	A meaner woman was delivered
	Of such a burden, male twins, both alike.
Thou drunken slave, I sent thee for a rope;
	And told thee to what purpose, and what end.
Come on, sir knave, have done your foolishness,
	And tell me how thou hast dispos'd thy charge.
Nell, sir; but her name and three quarters,—that is, an ell and three quarters,—will not measure her from hip to hip.
Master, be wise: an if you give it her,
	The devil will shake her chain and fright us with it.
I have but lean luck in the match, and yet is she a wondrous fat marriage.
Villain, thou liest; for even her very words
	Didst thou deliver to me on the mart.
Come to the Centaur; fetch our stuff from thence:
	I long that we were safe and sound aboard.
Master, there is a bark of Epidamnum
	That stays but till her owner comes aboard,
	And then she bears away.
Stop in your wind, sir: tell me this, I pray:
	Where have you left the money that I gave you?
Upon my life, by some device or other
	The villain is o'er-raught of all my money.
Marry, sir, besides myself, I am due to a woman; one that claims me, one that haunts me, one that will have me.
As good to wink, sweet love, as look on night.
No, trust me, sir, not I.



	I am sure thou dost.
I did obey, and sent my peasant home
	For certain ducats: he with none return'd.
Hath he not lost much wealth by wrack of sea?
Go get thee gone: fetch me an iron crow.
If thy name be call'd Luce,—Luce, thou hast answer'd him well.
Come, go along; my wife is coming yonder.
Sirrah, if any ask you for your master,
	Say he dines forth, and let no creature enter.
Why, thou didst conclude hairy men plain dealers without wit.
Unless the fear of death doth make me dote,
	I see my son Antipholus and Dromio!
Wast thou mad,

	That thus so madly thou didst answer me?
The duke and all that know me in the city
	Can witness with me that it is not so:
	I ne'er saw Syracusa in my life.
O yes; if any hour meet a sergeant, a' turns back for very fear.
Master, if you do, expect spoon-meat, so bespeak a long spoon.
'Tis so, I am an ass; else it could never be
	But I should know her as well as she knows me.
The fellow is distract, and so am I;
	And here we wander in illusions:
	Some blessed power deliver us from hence!
I'll to the Centaur, to go seek this slave:
	I greatly fear my money is not safe.
No, sir, 'tis in grain; Noah's flood could not do it.
Neither my husband, nor the slave return'd,
	That in such haste I sent to seek his master!
Go back again, and be new beaten home?
Back, slave, or I will break thy pate across.
Go bear it to the Centaur, where we host,
	And stay there, Dromio, till I come to thee.
I see, a man here needs not live by shifts,
	When in the streets he meets such golden gifts.
Thou gaoler, thou,
	I am thy prisoner: wilt thou suffer them
	To make a rescue?
My wife is in a wayward mood to-day,
	And will not lightly trust the messenger.
Ay; and let none enter, lest I break your pate.
Marry, he must have a long spoon that must eat with the devil.
I'll to the mart, and there for Dromio stay:
	If any ship put out, then straight away.
Till he come home again, I would forbear.
but I think him better than I say,
	And yet would herein others' eyes were worse.
Why, but there's many a man hath more hair than wit.
And, gentle master, I receiv'd no gold;
	But I confess, sir, that we were lock'd out.
Good meat, sir, is common; that every churl affords.
let me say no more;
	Gather the sequel by that went before.
I am your master, Dromio:
	Come, go with us; we'll look to that anon:
	Embrace thy brother there; rejoice with him.
Ay, sir; but I am sure I do not; and whatsoever a man denies, you are now bound to believe him.
When the sun shines let foolish gnats make sport,
	But creep in crannies when he hides his beams.
But say, sir, is it dinner-time?
I did not, gentle husband, lock thee forth.
Horn-mad, thou villain!
Marry, and did, sir; namely, no time to recover hair lost by nature.
And he will bless that cross with other beating:
	Between you, I shall have a holy head.
Run, master, run; for God's sake, take a house!
I would I were senseless, sir, that I might not feel your blows.
Why is Time such a niggard of hair, being, as it is, so plentiful an excrement?
God and the rope-maker bear me witness
	That I was sent for nothing but a rope!
Keep then fair league and truce with thy true bed;
	I live unstain'd, thou undishonoured.
Good Master doctor, see him safe convey'd
	Home to my house.
I pray you, jest, sir, as you sit at dinner.
No, sir: I think the meat wants that I have



	In good time, sir; what's that?
For God's sake, send some other messenger.
Because it is a blessing that he bestows on beasts: and what he hath scanted men in hair, he hath given them in wit.
But, I pray, sir, why am I beaten?
It may be so, but I did never see it.
Go back again, thou slave, and fetch him home.
No; bear it with you, lest I come not time enough.
Sir, sooth to say, you did not dine at home.
You have prevail'd: I will depart in quiet,
	And, in despite of mirth, mean to be merry.
They must be bound and laid in some dark room.
Farewell till then: I will go lose myself,
	And wander up and down to view the city.
I think I had: I never did deny it.
Quoth my master:
	'I know,' quoth he, 'no house, no wife, no mistress.'
Speak softly: yonder, as I think, he walks.
I see, sir, you have found the goldsmith now:
	Is that the chain you promis'd me to-day?
The plainer dealer, the sooner lost: yet be loseth it in a kind of jollity.
I know it well, sir: lo, here is the chain.
Come, Dromio, come; these jests are out of season;
	Reserve them till a merrier hour than this.
To none of these, except it be the last;
	Namely, some love that drew him oft from home.
I pray you, sir, my ring, or else the chain:
	I hope you do not mean to cheat me so.
Go, some of you, knock at the abbey gate
	And bid the lady abbess come to me.
She is a virtuous and a reverend lady:
	It cannot be that she hath done thee wrong.
Yes, to pay a fine for a periwig and recover the lost hair of another man.
Well, I will marry one day, but to try.
An if I have not, sir, I hope you have,
	Or else you may return without your money.
Hopeless and helpless doth Ægeon wend,
	But to procrastinate his lifeless end.
I think thou art an ass.
Some get within him, take his sword away.
I do obey thee till I give thee bail.
Time is a very bankrupt, and owes more than he's worth to season.
wilt thou flout me thus unto my face,
	Being forbid?
So come, help: well struck!
I am not furnish'd with the present money;
	Besides, I have some business in the town.
I am an ass indeed; you may prove it by my long ears.
too soon
	We came aboard.
had the gods done so, I had not now
	Worthily term'd them merciless to us!
Come, come, Antipholus; we dine too late.
Brought to this town by that most famous warrior,
	Duke Menaphon, your most renowned uncle.
Thou art, as you are all, a sorceress:
	I conjure thee to leave me and be gone.
I should have child you for not bringing it,
	But, like a shrew, you first begin to brawl.
Stay, stand apart; I know not which is which.
I, sir, am Dromio: pray let me stay.
Ay, ay, he told his mind upon mine ear.
Come, come; you know I gave it you even now.
Let us come in, that we may bind him fast,
	And bear him home for his recovery.
Did not her kitchen-maid rail, taunt, and scorn me?
It is a branch and parcel of mine oath,
	A charitable duty of my order;
	Therefore depart and leave him here with me.
Well, sir, then 'twill be dry.
Thou art sensible in nothing but blows, and so is an ass.
Sister, go you with me.—



	Say now, whose suit is he arrested at?
If thou art chang'd to aught, 'tis to an ass.
I think I did, sir; I deny it not.
that is where we din'd,
	Where Dowsabel did claim me for her husband:
	She is too big, I hope, for me to compass.
Thou whoreson, senseless villain!
How chance thou art return'd so soon?
Let's call more help
	To have them bound again.
As from a bear a man would run for life,
	So fly I from her that would be my wife.
Methinks you are my glass, and not my brother:
	I see by you I am a sweet-fac'd youth.
Come, stand by me; fear nothing.
Buried some dear friend?
Say, wherefore didst thou lock me forth to-day?
If you went in pain, master, this 'knave' would go sore.
Either get thee from the door or sit down at the hatch.
His company must do his minions grace,
	Whilst I at home starve for a merry look.
Hie thee, slave, be gone!
No evil lost is wail'd when it is gone.
Now, out of doubt, Antipholus is mad,
	Else would he never so demean himself.
Come, sister; I am press'd down with conceit; Conceit, my comfort and my injury.
Mights thou perceive austerely in his eye
	That he did plead in earnest?
Is there any ship puts forth to-night?
Even now, even here, not half-an-hour since.
Not mad, but mated; how, I do not know.
I think you are all mated or stark mad.
That I should be attach'd in Ephesus,
	I tell you, 'twill sound harshly in her ears.
Go bind this man, for he is frantic too.
Sure, Luciana, it is two o'clock.
Beyond imagination is the wrong
	That she this day hath shameless thrown on me.
To what end did I bid thee hie thee home?
Faith, no; he comes too late;
	And so tell your master.
be mad, good master; cry, 'the devil!'
They stand at the door, master: bid them welcome hither.
Be it my wrong you are from me exempt,
	But wrong not that wrong with a more contempt.
thy master and his man are here,
	And that is false thou dost report to us.
I did not see you since you sent me hence,
	Home to the Centaur, with the gold you gave me.
These ears of mine, thou know'st, did hear thee.
I know thou canst; and therefore, see thou do it.
Am I in earth, in heaven, or in hell?
quoth he:
	'My mistress, sir,' quoth I: 'hang up thy mistress!
Shall love, in building, grow so ruinous?
This day, great duke, she shut the doors upon me,
	While she with harlots feasted in my house.
To a rope's end, sir; and to that end am I return'd.
Perhaps some merchant hath invited him,
	And from the mart he's somewhere gone to dinner.
We being strangers here, how dar'st thou trust
	So great a charge from thine own custody?
What observation mad'st thou in this case
	Of his heart's meteors tilting in his face?
Since that my beauty cannot please his eye,
	I'll weep what's left away, and weeping die.
Well, sir, I will.
The one, to save the money that he spends in tiring; the other, that at dinner they should not drop in his porridge.
But your reason was not substantial, why there is not time to recover.
Say, is your tardy master now at hand?
I will debate this matter at more leisure,
	And teach your ears to list me with more heed.
That's not my fault; he's master of my state:
	What ruins are in me that can be found
	By him not ruin'd?
Our fraughtage, sir,
	I have convey'd aboard, and I have bought
	The oil, the balsamum, and aqua-vitæ.
quoth he:
	'Where is the thousand marks I gave thee, villain?'
I do not know the matter: he is 'rested on the case.
Small cheer and great welcome makes a merry feast.
I would not spare my brother in this case,
	If he should scorn me so apparently.
He had, my lord; and when he ran in here,
	These people saw the chain about his neck.
Saw'st thou him enter at the abbey here?
I owe you none till I receive the chain.
Thither I must, although against my will,
	For servants must their masters' minds fulfil.
I sent you money to redeem you,
	By Dromio here, who came in haste for it.
What patch is made our porter?—My master stays in the street.
I hear him, mistress: fly, be gone!
I am not mad.
There's no time for a man to recover his hair that grows bald by nature.
no marvel though she pause;
	They can be meek that have no other cause.
Justice, most gracious duke!
Good people, enter, and lay hold on him.
A sin prevailing much in youthful men,
	Who give their eyes the liberty of gazing.
I see two husbands, or mine eyes deceive me!
Not a man of those but he hath the wit to lose his hair.
I am an ass, I am a woman's man and besides myself.
Beshrew his hand, I scarce could understand it.
I never saw you in my life till now.
O most unhappy day!
He that commends me to mine own content,
	Commends me to the thing I cannot get.
I did not look so low.
Wilt thou still talk?
It seems, his sleeps were hinder'd by thy railing,
	And thereof comes it that his head is light.
You are a merry man, sir: fare you well.
he is mad.
You wrong me more, sir, in denying it:
	Consider how it stands upon my credit.
Call thyself sister, sweet, for I aim thee.
I am transformed, master, am not I?
Arrest me, foolish fellow, if thou dar'st.
I think it be, sir; I deny it not.
Either consent to pay this sum for me,
	Or I attach you by this officer.
is not your husband mad?
No, my good lord: myself, he, and my sister
	To-day did dine together.
Shall, Antipholus,
	Even in the spring of love, thy love-springs rot?
The hour steals on; I pray you, sir, dispatch.
Thou drunkard, thou, what didst thou mean by this?
Master, knock the door hard.
How can she thus then, call us by our names,
	Unless it be by inspiration?
Ay, but not rough enough.
Good sir, say whe'r you'll answer me or no:
	If not, I'll leave him to the officer.
There's none but asses will be bridled so.
He did bespeak a chain for me, but had it not.
that did I never hear.
Here comes your man: now is your husband nigh.
With words that in an honest suit might move.
Therefore, give out you are of Epidamnum,
	Lest that your goods too soon be confiscate.
I never saw my father in my life.
thou speak'st false in both.
What art thou that keep'st me out from the house I owe?
But soft!
This course I fittest choose,
	For forty ducats is too much to lose.
Methinks your maw, like mine, should be your clock
	And strike you home without a messenger.
Saving your merry humour, here's the note
	How much your chain weighs to the utmost carat.
Nay, not sure, in a thing falsing.
O most unhappy strumpet!
He is my prisoner: if I let him go,
	The debt he owes will be requir'd of me.
Break any breaking here, and I'll break your knave's pate.
Yet this my comfort: when your words are done,
	My woes end likewise with the evening sun.
bind him, bind him, let him not come near me.
Good sir, be patient.
We'll draw cuts for the senior: till then lead thou first.
Ere I learn love, I'll practise to obey.
Hast thou delight to see a wretched man
	Do outrage and displeasure to himself?
She never reprehended him but mildly
	When he demean'd himself rough, rude, and wildly.
You gave me none: you wrong me much to say so.
As sure, my liege, as I do see your Grace.
Go call the abbess hither.
I, sir, am Dromio: command him away.
Since mine own doors refuse to entertain me,
	I'll knock elsewhere, to see if they'll disdain me.
Mome, malt-horse, capon, coxcomb, idiot, patch!
Hath homely age the alluring beauty took
	From my poor cheek?
I never saw her till this time.
I know not thy mistress: out on thy mistress!'
'Tis so; and that self chain about his neck
	Which he forswore most monstrously to have.
Whoever bound him, I will loose his bonds,
	And gain a husband by his liberty.
I am not in a sportive humour now.
I will determine this before I stir.
Master, shall I be porter at the gate?
the fiend is strong within him.
Sure these are but imaginary wiles,
	And Lapland sorcerers inhabit here.
is he well?
Marry, so it doth appear
	By the wrongs I suffer and the blows I bear.
is your merry humour alter'd?
Well, sir, there rest in your foolery.
You would all this time have proved there is no time for all things.
Those,—for their parents were exceeding poor,—
	I bought, and brought up to attend my sons.
Look'd he or red or pale?
Master, shall I fetch your stuff from shipboard?
What wilt thou do, thou peevish officer?
Or sleep I now and think I hear all this?
Nothing, sir, but that I am beaten.
Yea, dost thou jeer, and flout me in the teeth?
So befall my soul
	As this is false he burdens me withal!
Why, sir, I gave the money for the rope.
No, sir, not I; I came from Syracuse.
But say, I prithee, is he coming home?
I think you all have drunk of Circe's cup.
Were not my doors lock'd up and I shut out?
Good sir, draw near to me, I'll speak to him.
I knew he was not in his perfect wits.
have you got the picture of old Adam new apparelled?
This fool-begg'd patience in thee will be left.
And may it be that you have quite forgot A husband's office?
I'll meet you at that place some hour hence.
Return'd so soon!
On, officer, to prison till it come.
How many fond fools serve mad jealousy!
Come, Dromio, let us go.
Dromio, what stuff of mine hast thou embark'd?
Far from her nest the lapwing cries away:
	My heart prays for him, though my tongue do curse.
It seems he hath great care to please his wife.
Spake he so doubtfully, thou couldst not feel his meaning?
Then, let your servants bring my husband forth.
Is not your name, sir, called Antipholus?




















	Proceed, Solinus, to procure my fall,
	And by the doom of death end woes and all.
Master, I am here enter'd in bond for you.
Here comes my man: I think he brings the money.
Consent to pay thee that I never had!
Nay, not sound, I pray you.
I should kick, being kick'd; and, being at that pass,
	You would keep from my heels and beware of an ass.
poor man, how pale and wan he looks!
that thou wert not, poor distressed soul!
I understand thee not.
Hold, sir, for God's sake!
I must laugh.
One of these men is Genius to the other;
	And so of these: which is the natural man,
	And which the spirit?
God help, poor souls!
I bespoke it not



	Not once, nor twice, but twenty times you have.
Who would be jealous then, of such a one?
And did not I in rage depart from thence?
Until I know this sure uncertainty,
	I'll entertain the offer'd fallacy.
Ægeon art thou not?
or sad or merrily?
I see thy age and dangers make thee dote.
Think'st thou I jest?
If I should pay your worship those again,
	Perchance you will not bear them patiently.
For gazing on your beams; fair sun, being by.
Nay, an you will not, sir, I'll take my heels.
I knew 'twould be a bald conclusion.
if thou be'st the same Ægeon, speak,
	And speak unto the same Æmilia!
But, were you wedded, you would bear some sway.
who wafts us yonder?
Then swore he that he was a stranger here.
What, will you walk with me about the town,
	And then go to my inn and dine with me?
Nor to-day here you must not; come again when you may.
That's a fault that water will mend.
Hapless Ægeon, whom the fates have mark'd
	To bear the extremity of dire mishap!
If it be, sir, I pray you eat none of it.
Will you send him, mistress, redemption, the money in his desk?
I came from Corinth, my most gracious lord,—



	And I with him.
You know I gave it you half an hour since.
Why prat'st thou to thyself and answer'st not?
Hath not else his eye
	Stray'd his affection in unlawful love?
Ay, but not enough.
Go borrow me a crow.
I hold your dainties cheap, sir, and your welcome dear.
mad or well-advis'd?
Master, is this Mistress Satan?
He meant he did me none; the more my spite.
I found it by the barrenness; hard in the palm of the hand.
I do arrest you, sir: you hear the suit.
Lest it make you choleric, and purchase me another dry basting.
But had he such a chain of thee, or no?
Why, thou peevish sheep,
	What ship of Epidamnum stays for me?
You are not Pinch's patient, are you, sir?
Why, headstrong liberty is lash'd with woe.
Nay, 'tis for me to be patient; I am in adversity.
There, take it; and much thanks for my good cheer.
Who talks within there?
I pray you, master, tell me.
Be quiet, and depart: thou shalt not have him.
Husband, I'll dine above with you to-day,
	And shrive you of a thousand idle pranks.
Thou villain, what say'st thou?
I am sorry now that I did draw on him.
Have at you with a proverb: Shall I set in my staff?
I know his eye doth homage otherwhere,
	Or else what lets it but he would be here?
There is my hand, and let it feel your ear.
Here's too much 'out upon thee!'
By my troth your town is troubled with unruly boys.
Well, sir, I thank you.
It shall not need: thy father hath his life.
May he not do it by fine and recovery?
quoth he:
	'Your meat doth burn,' quoth I; 'my gold!'
tell me at whose suit.
Go bid them let us in.
Good sir, make haste.
O mistress, mistress!
No, he's in Tartar limbo, worse than hell.
'Tis true; she rides me and I long for grass.
Antipholus, thou cam'st from Corinth first?
Say, woman, didst thou so?
Here's that, I warrant you, will pay them all.
I am glad to see you in this merry vein:
	What means this jest?
Let's hear it.
you might have come before.
Five hundred ducats, villain, for a rope?
Go fetch me something: I'll break ope the gate.
Your goods that lay at host, sir, in the Centaur.
Pleaseth you walk with me down to his house,
	I will discharge my bond, and thank you too.
A grievous fault!
wherefore dost thou mad me?
Am I so round with you as you with me,
	That like a football you do spurn me thus?
Marry, sir, in her buttocks: I found it out by the bogs.
Well, officer, arrest him at my suit.
I pray you, let me see it.
Dromio, play the porter well.
Kneel to the duke before he pass the abbey.
You would say so, master, if your garments were thin.
Dromio, go bid the servants spread for dinner



	O, for my beads!
Thou hast suborn'd the goldsmith to arrest me.
for my dinner: I have not din'd to-day.
I pray thee, let me in.
Sure ones then.
Nay, he's at two hands with me, and that my two ears can witness.
Satan, avoid!
To fetch my poor distracted husband hence.
how comes it,
	That thou art thus estranged from thyself?
Certain ones, then.
See where they come: we will behold his death.
Thou baggage, let me in.
As if Time were in debt!
Do you not hear it ring?
First, he did praise my beauty, then my speech.
It is no shame: the fellow finds his vein,
	And, yielding to him humours well his frenzy.
Your mistress sent to have me home to dinner?
That labour may you save: see where he comes.
See, here he comes.
Speak freely, Syracusian, what thou wilt.
my old master; who hath bound him here?
Right, sir; I'll tell you when, an you'll tell me wherefore.
It seems thou wantest breaking: out upon thee, hind!
Her part, poor soul!
now your jest is earnest.
I'll say as they say, and persever so,
	And in this mist at all adventures go.
Have patience, sir; O!
quoth he:
	'Will you come home?'
Tell her I am arrested in the street,
	And that shall bail me.
I, gentle mistress.
This jest shall cost me some expense.
I see, these witches are afraid of swords.
Hence, prating peasant!
Dost thou not know?
Well, I'll break in.
'The pig,' quoth I, 'is burn'd;' 'my gold!'
In debating which was best, we shall part with neither.
my door is lock'd.
then, he hath wasted it:
	Are my discourses dull?
Ay, sir, and wherefore; for they say every why hath a wherefore.
beat it hence.
hurt him not, for God's sake!
Thou hast thine own form.
There is your money that I had to keep.
Din'd at home!
Sir, I commend you to your own content.
Come not near her.
Discover how, and thou shalt find me just.
Avoid thee, fiend!
Give me your hand and let me feel your pulse.
The porter for this time, sir, and my name is Dromio.
Your wife, sir knave!
Money by me!
Why look you strange on me?
I cross me for a sinner.
Not I, sir; you are my elder.
In verity you did: my bones bear witness,
	That since have felt the vigour of his rage.
Tell me, and dally not, where is the money?
Justice, most sacred duke, against the abbess!
How long hath this possession held the man?
Who heard me to deny it or forswear it?
I'll serve you, sir, five hundred at the rate.
Ay, when fowls have no feathers, and fish have no fin.
First he denied you had in him no right.
When were you wont to use my sister thus?
'Tis true, my liege; this ring I had of her.
Masters, let him go:
	He is my prisoner, and you shall not have him.
what mistress, slave, hast thou?
As you love strokes, so jest with me again.
Ourselves we do remember, sir, by you;
	For lately we were bound, as you are now.
Hark, hark!
sweet, now, make haste.
I do;
	And charge you in the duke's name to obey me.
Why, so I did.
I durst have denied that, before you were so choleric.
I charge thee tempt me not!
Let my master in, Luce.
How dost thou mean a fat marriage?
Sir, I must have that diamond from you.
I never spake with her in all my life.
Where is the gold I gave in charge to thee?
Made it for me, sir!
As roughly as my modesty would let me.
Out on thee, villain!
Because their business still lies out o' door.
No; not a creature enters in my house.
Why, first,—for flouting me; and then, wherefore,—
	For urging it the second time to me.
Let him knock till it ache.
What, thou meanest an officer?
Why, give it to my wife and fetch your money.
Have patience, I beseech.
Your man and you are marvellous merry, sir.
Fie, brother: how the world is chang'd with you!
And this is false you burden me withal.
may we be gone?
Look, when I serve him so, he takes it ill.



	O!
Come, sir, to dinner.
No, none by me.
By what rule, sir?
With all my heart I'll gossip at this feast.
There, take you that, sir knave.
No, I am an ape.
I dare, and do defy thee for a villain.
I am sure you both of you remember me.
Away!
What a coil is there, Dromio!
Neither.
'Fly pride,' says the peacock: mistress, that you know.
Come, sister.
What woman's man?
Master, here's the gold you sent me for.
This purse of ducats I receiv'd from you,
	And Dromio, my man, did bring them me.
fetch thy master home.
Then fairly I bespoke the officer
	To go in person with me to my house.
In her forehead; armed and reverted, making war against her heir.
And why dost thou deny the bag of gold?
Justice, sweet prince, against that woman there!
I, sir?
Yes, that you did, sir, and forswore it too.
how fondly dost thou reason!
Know'st thou his mind?
How is the man esteem'd here in the city?
Went'st not thou to her for a purse of ducats?
He did, and from my finger snatch'd that ring.
Did you converse, sir, with this gentle-woman?
And welcome more common, for that's nothing but words.
would you create me new?
A crow without feather?
O villain!
Here is thy fee: arrest him, officer.
What please yourself, sir: I have made it for you.
Luciana, did he tempt thee so?
for God's sake, hold your hands!
I thought to have ask'd you.
Transform me then, and to your power I'll yield.
when spake I such a word?
I know the man.
One Angelo, a goldsmith; do you know him?
What is the sum he owes?
He came to me, and I deliver'd it.
Thank me, sir!
Wherefore throng you hither?
Get thee away.
This is some priory: in, or we are spoil'd.
By running fast.
Nay, rather persuade him to hold his hands.
What Adam dost thou mean?
where run'st thou so fast?
Merchant of Syracusa, plead no more.
Haply, in private.
How if your husband start some other where?
Go bear him hence.
O, sir!
Why call you me love?
What then became of them, I cannot tell;
	I to this fortune that you see me in.
Unfeeling fools can with such wrengs dispense.
Be quiet, people.
Peace, doting wizard, peace!
I see the goldsmith.
Perdy, your doors were lock'd and you shut out.
Thy sister's sister.
Why, man, what is the matter?
Peace, fool!
How hast thou lost thy breath?
Why bear you these rebukes and answer not?
Dissembling villain!
She sent for you by Dromio home to dinner.
you know me well.
And to that end, sir, I will welcome you.
they are loose again.
Why, this is strange.
Ay me, it is my husband!
They are both forsworn:
	In this the madman justly chargeth them!
For two; and sound ones too.
Say, how grows it due?
By me?
Due for a chain your husband had of him.
Here, go: the desk!
She did betray me to my own reproof.
Do you know me, sir?
Good now, hold thy tongue.
Where is the thousand marks thou hadst of me?
Fie on thee, wretch!
There is something in the wind, that we cannot get in.
know he is the bridle of your will.
why, you gave no gold to me.
You'll cry for this, minion, if I beat the door down.
now you run this humour out of breath.
What simple thief brags of his own attaint?
Certes, she did; the kitchen-vestal scorn'd you.
Will you walk in to see their gossiping?
Is't good to soothe him in these contraries?
Which of you two did dine with me to-day?
there was blow for blow.
The hours come back!
am I your man?
O perjur'd woman!
call my sister so.
Then she bears some breadth?
Upon what bargain do you give it me?
Say, didst thou speak with him?
I will, my lord.
Why, what an intricate impeach is this!
I'll make you amends next, to give you nothing for something.
are you mad, that you do reason so?
Who is that at the door that keeps all this noise?
Good Lord!
Will you be bound for nothing?
Not know my voice!
Not this, but troubles of the marriage-bed.
Sir, he din'd with her there, at the Porpentine.
With what persuasion did he tempt thy love?
What, is he arrested?
Dissembling harlot!
What answer, sir?
Can you tell for whose sake?
In what part of her body stands Ireland?
Go, get thee from the door.
barren my wit?
What needs all that, and a pair of stocks in the town?
Free from these slanders and this open shame!
what tell'st thou me of supping?
you'll let us in, I trow?
Are you there, wife?
It is a fault that springeth from your eye.
That is the chain, sir, which you had of me.
O Lord!
A ship you sent me to, to hire waftage.
Master, mean you so?
I have served him from the hour of my nativity to this instant, and have nothing at his hands for my service but blows.
Shall I tell you why?
Plead you to me, fair dame?
It is the devil.
Bind Dromio too, and bear them to my house.
His incivility confirms no less.
What error drives our eyes and ears amiss?
Avaunt, thou witch!
You hear how he importunes me: the chain!
This is the fairy land: O!
open the door.
a madman!
And in assemblies too.
How comes it now, my husband, O!
Sconce, call you it?
That love I begg'd for you he begg'd of me.
No; I say nay to that.
Give me thy hand.
yea or no?
The money that you owe me for the chain.
And come with naked swords.
More company!
Come, where's the chain?
He cries for you and vows, if he can take you,
	To scotch your face, and to disfigure you.
Two hundred ducats.
Do you hear, you minion?
These ducats pawn I for my father here.
Here comes the almanack of my true date.
And is not that your bondman Dromio?
I answer you!
All this my sister is, or else should be.
But where's the money?
Dromio, keep the gate.
Then, you will bring the chain to her yourself?
This chain you had of me; can you deny it?
Self-harming jealousy!
how fiery and how sharp he looks.
can you tell?
how idly do they talk.
we'll mend our dinner here.
Complain unto the duke of this indignity.
who are those at the gate?
Gaze where you should, and that will clear your sight.
Marry, sir, for this something that you gave me for nothing.
then is he the ground
	Of my defeatures.
She that would be your wife now ran from you.
am I Dromio?
was I married to her in my dream?
And did not she herself revile me there?
You know no Centaur?
go, get you from the door.
Maud, Bridget, Marian, Cicely, Gillian, Ginn!
That's a question: how shall we try it?
Didst speak him fair?
Hold, take thou that, and that.
And you, sir, for this chain arrested me.
Where is thy master, Dromio?
Mark how he trembles in his ecstasy!
Which of these sorrows is he subject to?
Have you the chain about you?
Are you a god?
Sleeping or waking?
ho!
By Dromio?
How now, sir!
Will you go with me?
Thy mistress' marks!
Guard with halberds!
This servitude makes you to keep unwed.
What is your will that I shall do with this?
And are not you my husband?
He speaks to me.
Sirrah, what say you?
This touches me in reputation.
What, will you murder me?
And what said he?
My business cannot brook this dalliance.
quoth I: 'my gold!'
To me, sir?
Fie, how impatience loureth in your face!
What is the course and drift of your compact?
Basting.
am I myself?
Have at you with another: that's—when?
You receiv'd no gold?
What mean you, sir?
O!
Do so.
My house was at the Phœnix?
Gaoler, take him to thy custody.
Ay, that's my name.
What gold is this?
Master Antipholus!
Here are the angels that you sent for to deliver you.
Known unto these, and to myself disguis'd!
What's her name?
That's my sister.
they'll kill us.
And you said, no.
have you that I sent you for?
To me she speaks; she moves me for her theme!
Then pleaded I for you.
shift and save yourself!
What complexion is she of?
what should I answer you?
And I am witness with her that she did.
you use this dalliance to excuse
	Your breach of promise to the Porpentine.
Hold!
Why should their liberty than ours be more?
spite of spites.
Why, how now, Dromio!
What claim lays she to thee?
Where stood Belgia, the Netherlands?
How say you now?
Wherefore?
Patience unmov'd!
You minion, you, are these your customers?
Alas!
Name them.
Your reason?
the purse!
Sans fable, she herself revil'd you there.
Dromio, nor thou?
and how besides thyself?
Why, Dromio?
Who deciphers them?
No?
God, for thy mercy!
What is she?
or else his ghost?
Ay me!
Where America, the Indies?
Where England?
Quoth who?
Where Spain?
For what reason?
Where Scotland?
What now?
How now!
What, the chain?
Upon what cause?
Where France?
You should for that have reprehended him.


"""
meteor_scores, bert_scores, rouge_scores = evaluate_summaries(generated_summary, reference_summary)

# Print evaluation scores
print("METEOR scores:")
for method, score in meteor_scores.items():
    print(f"{method}: {score}")
print()

print("BERT scores:")
for method, score in bert_scores.items():
    print(f"{method}: {score}")
print()

print("ROUGE scores:")
for method, score in rouge_scores.items():
    print(f"{method}: {score}")



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


METEOR scores:
Generated: 0.162314287584266

BERT scores:
Generated: 0.7782830595970154

ROUGE scores:
Generated: {'rouge-1': {'r': 0.4354243542435424, 'p': 0.05361199454793276, 'f': 0.09546925371123906}, 'rouge-2': {'r': 0.10365853658536585, 'p': 0.007945162797943606, 'f': 0.014759078405515678}, 'rouge-l': {'r': 0.42435424354243545, 'p': 0.052248977737392094, 'f': 0.09304206924521963}}


In [ ]:
#2
#Weighting
generated_summary="""Proceed, Solinus, to procure my fall,
	And by the doom of death end woes and all.
Merchant of Syracusa, plead no more.
Yet this my comfort: when your words are done,
	My woes end likewise with the evening sun.
In Syracusa was I born, and wed
	Unto a woman, happy but for me,
	And by me too, had not our hap been bad.
That very hour, and in the self-same inn,
	A meaner woman was delivered
	Of such a burden, male twins, both alike.
Those,—for their parents were exceeding poor,—
	I bought, and brought up to attend my sons.
My wife, not meanly proud of two such boys,
	Made daily motions for our home return:
	Unwilling I agreed; alas!
too soon
	We came aboard.
let me say no more;
	Gather the sequel by that went before.
Nay, forward, old man; do not break off so;
	For we may pity, though not pardon thee.
had the gods done so, I had not now
	Worthily term'd them merciless to us!
Her part, poor soul!
Hapless Ægeon, whom the fates have mark'd
	To bear the extremity of dire mishap!
Gaoler, take him to thy custody.
I will, my lord.
Hopeless and helpless doth Ægeon wend,
	But to procrastinate his lifeless end.
Therefore, give out you are of Epidamnum,
	Lest that your goods too soon be confiscate.
There is your money that I had to keep.
Go bear it to the Centaur, where we host,
	And stay there, Dromio, till I come to thee.
Get thee away.
Many a man would take you at your word,
	And go indeed, having so good a mean.
What, will you walk with me about the town,
	And then go to my inn and dine with me?
I am invited, sir, to certain merchants,
	Of whom I hope to make much benefit;
	I crave your pardon.
Farewell till then: I will go lose myself,
	And wander up and down to view the city.
Sir, I commend you to your own content.
He that commends me to mine own content,
	Commends me to the thing I cannot get.
Here comes the almanack of my true date.
How chance thou art return'd so soon?
Return'd so soon!
Stop in your wind, sir: tell me this, I pray:
	Where have you left the money that I gave you?
I am not in a sportive humour now.
Tell me, and dally not, where is the money?
We being strangers here, how dar'st thou trust
	So great a charge from thine own custody?
I pray you, jest, sir, as you sit at dinner.
Methinks your maw, like mine, should be your clock
	And strike you home without a messenger.
Come, Dromio, come; these jests are out of season;
	Reserve them till a merrier hour than this.
Where is the gold I gave in charge to thee?
To me, sir?
why, you gave no gold to me.
Come on, sir knave, have done your foolishness,
	And tell me how thou hast dispos'd thy charge.
Where is the thousand marks thou hadst of me?
If I should pay your worship those again,
	Perchance you will not bear them patiently.
Thy mistress' marks!
what mistress, slave, hast thou?
wilt thou flout me thus unto my face,
	Being forbid?
There, take you that, sir knave.
What mean you, sir?
for God's sake, hold your hands!
Nay, an you will not, sir, I'll take my heels.
Upon my life, by some device or other
	The villain is o'er-raught of all my money.
I'll to the Centaur, to go seek this slave:
	I greatly fear my money is not safe.
Neither my husband, nor the slave return'd,
	That in such haste I sent to seek his master!
Sure, Luciana, it is two o'clock.
Perhaps some merchant hath invited him,
	And from the mart he's somewhere gone to dinner.
Why should their liberty than ours be more?
Because their business still lies out o' door.
Look, when I serve him so, he takes it ill.



	O!
know he is the bridle of your will.
There's none but asses will be bridled so.
Why, headstrong liberty is lash'd with woe.
This servitude makes you to keep unwed.
Not this, but troubles of the marriage-bed.
But, were you wedded, you would bear some sway.
Ere I learn love, I'll practise to obey.
How if your husband start some other where?
Till he come home again, I would forbear.
Patience unmov'd!
no marvel though she pause;
	They can be meek that have no other cause.
This fool-begg'd patience in thee will be left.
Well, I will marry one day, but to try.
Here comes your man: now is your husband nigh.
Say, is your tardy master now at hand?
Nay, he's at two hands with me, and that my two ears can witness.
Say, didst thou speak with him?
Know'st thou his mind?
Ay, ay, he told his mind upon mine ear.
Beshrew his hand, I scarce could understand it.
Spake he so doubtfully, thou couldst not feel his meaning?
But say, I prithee, is he coming home?
It seems he hath great care to please his wife.
Why, mistress, sure my master is horn-mad.
Horn-mad, thou villain!
I mean not cuckold-mad; but, sure, he is stark mad.
quoth he:
	'Your meat doth burn,' quoth I; 'my gold!'
quoth he:
	'Will you come home?'
quoth I: 'my gold!'
quoth he:
	'Where is the thousand marks I gave thee, villain?'
'The pig,' quoth I, 'is burn'd;' 'my gold!'
quoth he:
	'My mistress, sir,' quoth I: 'hang up thy mistress!
I know not thy mistress: out on thy mistress!'
Quoth who?
Quoth my master:
	'I know,' quoth he, 'no house, no wife, no mistress.'
Go back again, thou slave, and fetch him home.
Go back again, and be new beaten home?
For God's sake, send some other messenger.
Back, slave, or I will break thy pate across.
And he will bless that cross with other beating:
	Between you, I shall have a holy head.
Hence, prating peasant!
fetch thy master home.
Am I so round with you as you with me,
	That like a football you do spurn me thus?
You spurn me hence, and he will spurn me hither:
	If I last in this service, you must case me in leather.
Fie, how impatience loureth in your face!
His company must do his minions grace,
	Whilst I at home starve for a merry look.
Hath homely age the alluring beauty took
	From my poor cheek?
then, he hath wasted it:
	Are my discourses dull?
barren my wit?
That's not my fault; he's master of my state:
	What ruins are in me that can be found
	By him not ruin'd?
Self-harming jealousy!
fie!
beat it hence.
Unfeeling fools can with such wrengs dispense.
I know his eye doth homage otherwhere,
	Or else what lets it but he would be here?
Since that my beauty cannot please his eye,
	I'll weep what's left away, and weeping die.
How many fond fools serve mad jealousy!
By computation, and mine host's report,
	I could not speak with Dromio since at first
	I sent him from the mart.
See, here he comes.
How now, sir!
is your merry humour alter'd?
As you love strokes, so jest with me again.
You know no Centaur?
You receiv'd no gold?
Your mistress sent to have me home to dinner?
My house was at the Phœnix?
Wast thou mad,

	That thus so madly thou didst answer me?
What answer, sir?
when spake I such a word?
Even now, even here, not half-an-hour since.
I did not see you since you sent me hence,
	Home to the Centaur, with the gold you gave me.
I am glad to see you in this merry vein:
	What means this jest?
I pray you, master, tell me.
Yea, dost thou jeer, and flout me in the teeth?
Think'st thou I jest?
Hold, take thou that, and that.
Hold, sir, for God's sake!
now your jest is earnest.
Upon what bargain do you give it me?
When the sun shines let foolish gnats make sport,
	But creep in crannies when he hides his beams.
Sconce, call you it?
But, I pray, sir, why am I beaten?
Dost thou not know?
Nothing, sir, but that I am beaten.
Shall I tell you why?
Ay, sir, and wherefore; for they say every why hath a wherefore.
Why, first,—for flouting me; and then, wherefore,—
	For urging it the second time to me.
Was there ever any man thus beaten out of season,
	When, in the why and the wherefore is neither rime nor reason?
Well, sir, I thank you.
Thank me, sir!
Marry, sir, for this something that you gave me for nothing.
I'll make you amends next, to give you nothing for something.
But say, sir, is it dinner-time?
No, sir: I think the meat wants that I have



	In good time, sir; what's that?
Well, sir, then 'twill be dry.
If it be, sir, I pray you eat none of it.
Your reason?
Lest it make you choleric, and purchase me another dry basting.
Well, sir, learn to jest in good time: there's a time for all things.
I durst have denied that, before you were so choleric.
By what rule, sir?
Marry, sir, by a rule as plain as the plain bald pate of Father Time himself.
Let's hear it.
There's no time for a man to recover his hair that grows bald by nature.
May he not do it by fine and recovery?
Yes, to pay a fine for a periwig and recover the lost hair of another man.
Why is Time such a niggard of hair, being, as it is, so plentiful an excrement?
Because it is a blessing that he bestows on beasts: and what he hath scanted men in hair, he hath given them in wit.
Why, but there's many a man hath more hair than wit.
Not a man of those but he hath the wit to lose his hair.
Why, thou didst conclude hairy men plain dealers without wit.
The plainer dealer, the sooner lost: yet be loseth it in a kind of jollity.
For what reason?
For two; and sound ones too.
Nay, not sound, I pray you.
Sure ones then.
Nay, not sure, in a thing falsing.
Certain ones, then.
Name them.
The one, to save the money that he spends in tiring; the other, that at dinner they should not drop in his porridge.
You would all this time have proved there is no time for all things.
Marry, and did, sir; namely, no time to recover hair lost by nature.
But your reason was not substantial, why there is not time to recover.
Thus I mend it: Time himself is bald, and therefore to the world's end will have bald followers.
I knew 'twould be a bald conclusion.
But soft!
who wafts us yonder?
How comes it now, my husband, O!
how comes it,
	That thou art thus estranged from thyself?
Thyself I call it, being strange to me,
	That, undividable, incorporate,
	Am better than thy dear self's better part.
I know thou canst; and therefore, see thou do it.
Keep then fair league and truce with thy true bed;
	I live unstain'd, thou undishonoured.
Plead you to me, fair dame?
Fie, brother: how the world is chang'd with you!
When were you wont to use my sister thus?
She sent for you by Dromio home to dinner.
By Dromio?
Did you converse, sir, with this gentle-woman?
What is the course and drift of your compact?
I, sir?
I never saw her till this time.
Villain, thou liest; for even her very words
	Didst thou deliver to me on the mart.
I never spake with her in all my life.
How can she thus then, call us by our names,
	Unless it be by inspiration?
Be it my wrong you are from me exempt,
	But wrong not that wrong with a more contempt.
To me she speaks; she moves me for her theme!
was I married to her in my dream?
Or sleep I now and think I hear all this?
What error drives our eyes and ears amiss?
Until I know this sure uncertainty,
	I'll entertain the offer'd fallacy.
Dromio, go bid the servants spread for dinner



	O, for my beads!
This is the fairy land: O!
spite of spites.
Why prat'st thou to thyself and answer'st not?
Dromio, thou drone, thou snail, thou slug, thou sot!
I am transformed, master, am not I?
I think thou art, in mind, and so am I.



	Nay, master, both in mind and in my shape.
Thou hast thine own form.
If thou art chang'd to aught, 'tis to an ass.
'Tis true; she rides me and I long for grass.
'Tis so, I am an ass; else it could never be
	But I should know her as well as she knows me.
Come, sir, to dinner.
Dromio, keep the gate.
Husband, I'll dine above with you to-day,
	And shrive you of a thousand idle pranks.
Sirrah, if any ask you for your master,
	Say he dines forth, and let no creature enter.
Come, sister.
Dromio, play the porter well.
Am I in earth, in heaven, or in hell?
Sleeping or waking?
mad or well-advis'd?
Known unto these, and to myself disguis'd!
I'll say as they say, and persever so,
	And in this mist at all adventures go.
Master, shall I be porter at the gate?
Ay; and let none enter, lest I break your pate.
Come, come, Antipholus; we dine too late.
Thou drunkard, thou, what didst thou mean by this?
I think thou art an ass.
Marry, so it doth appear
	By the wrongs I suffer and the blows I bear.
I should kick, being kick'd; and, being at that pass,
	You would keep from my heels and beware of an ass.
You are sad, Signior Balthazar: pray God, our cheer
	May answer my good will and your good welcome here.
I hold your dainties cheap, sir, and your welcome dear.
O, Signior Balthazar, either at flesh or fish,
	A table-full of welcome makes scarce one dainty dish.
Good meat, sir, is common; that every churl affords.
And welcome more common, for that's nothing but words.
Small cheer and great welcome makes a merry feast.
my door is lock'd.
Go bid them let us in.
Maud, Bridget, Marian, Cicely, Gillian, Ginn!
Mome, malt-horse, capon, coxcomb, idiot, patch!
Either get thee from the door or sit down at the hatch.
Dost thou conjure for wenches, that thou call'st for such store,
	When one is one too many?
Go, get thee from the door.
What patch is made our porter?—My master stays in the street.
Let him walk from whence he came, lest he catch cold on's feet.
Who talks within there?
open the door.
Right, sir; I'll tell you when, an you'll tell me wherefore.
Wherefore?
for my dinner: I have not din'd to-day.
Nor to-day here you must not; come again when you may.
What art thou that keep'st me out from the house I owe?
The porter for this time, sir, and my name is Dromio.
O villain!
thou hast stolen both mine office and my name:
	The one ne'er got me credit, the other mickle blame.
If thou hadst been Dromio to-day in my place,
	Thou wouldst have chang'd thy face for a name, or thy name for an ass.
What a coil is there, Dromio!
who are those at the gate?
Let my master in, Luce.
Faith, no; he comes too late;
	And so tell your master.
O Lord!
I must laugh.
Have at you with a proverb: Shall I set in my staff?
Have at you with another: that's—when?
can you tell?
If thy name be call'd Luce,—Luce, thou hast answer'd him well.
Do you hear, you minion?
you'll let us in, I trow?
I thought to have ask'd you.
So come, help: well struck!
there was blow for blow.
Thou baggage, let me in.
Can you tell for whose sake?
Master, knock the door hard.
Let him knock till it ache.
You'll cry for this, minion, if I beat the door down.
What needs all that, and a pair of stocks in the town?
Who is that at the door that keeps all this noise?
By my troth your town is troubled with unruly boys.
Are you there, wife?
you might have come before.
Your wife, sir knave!
go, get you from the door.
If you went in pain, master, this 'knave' would go sore.
Here is neither cheer, sir, nor welcome: we would fain have either.
In debating which was best, we shall part with neither.
They stand at the door, master: bid them welcome hither.
There is something in the wind, that we cannot get in.
You would say so, master, if your garments were thin.
Your cake here is warm within; you stand here in the cold:
	It would make a man mad as a buck to be so bought and sold.
Go fetch me something: I'll break ope the gate.
Break any breaking here, and I'll break your knave's pate.
It seems thou wantest breaking: out upon thee, hind!
Here's too much 'out upon thee!'
I pray thee, let me in.
Ay, when fowls have no feathers, and fish have no fin.
Well, I'll break in.
Go borrow me a crow.
A crow without feather?
Master, mean you so?
For a fish without a fin, there's a fowl without a feather:
	If a crow help us in, sirrah, we'll pluck a crow together.
Go get thee gone: fetch me an iron crow.
Have patience, sir; O!
You have prevail'd: I will depart in quiet,
	And, in despite of mirth, mean to be merry.
Good sir, make haste.
Since mine own doors refuse to entertain me,
	I'll knock elsewhere, to see if they'll disdain me.
I'll meet you at that place some hour hence.
This jest shall cost me some expense.
And may it be that you have quite forgot A husband's office?
Shall, Antipholus,
	Even in the spring of love, thy love-springs rot?
Shall love, in building, grow so ruinous?
What simple thief brags of his own attaint?
Alas!
Against my soul's pure truth why labour you
	To make it wander in an unknown field?
Are you a god?
would you create me new?
Transform me then, and to your power I'll yield.
are you mad, that you do reason so?
Not mad, but mated; how, I do not know.
It is a fault that springeth from your eye.
For gazing on your beams; fair sun, being by.
Gaze where you should, and that will clear your sight.
As good to wink, sweet love, as look on night.
Why call you me love?
call my sister so.
Thy sister's sister.
That's my sister.
All this my sister is, or else should be.
Call thyself sister, sweet, for I aim thee.
Thee will I love and with thee lead my life:
	Thou hast no husband yet nor I no wife.
Give me thy hand.
soft, sir; hold you still:
	I'll fetch my sister, to get her good will.
Why, how now, Dromio!
where run'st thou so fast?
Do you know me, sir?
am I Dromio?
am I your man?
Thou art Dromio, thou art my man, thou art thyself.
I am an ass, I am a woman's man and besides myself.
What woman's man?
and how besides thyself?
Marry, sir, besides myself, I am due to a woman; one that claims me, one that haunts me, one that will have me.
What claim lays she to thee?
A very reverent body; aye, such a one as a man may not speak of, without he say, 'Sir-reverence.'
I have but lean luck in the match, and yet is she a wondrous fat marriage.
How dost thou mean a fat marriage?
That's a fault that water will mend.
No, sir, 'tis in grain; Noah's flood could not do it.
What's her name?
Nell, sir; but her name and three quarters,—that is, an ell and three quarters,—will not measure her from hip to hip.
Then she bears some breadth?
No longer from head to foot than from hip to hip: she is spherical, like a globe; I could find out countries in her.
In what part of her body stands Ireland?
Marry, sir, in her buttocks: I found it out by the bogs.
I found it by the barrenness; hard in the palm of the hand.
In her forehead; armed and reverted, making war against her heir.
Faith, I saw not; but I felt it hot in her breath.
O, sir!
Where stood Belgia, the Netherlands?
I did not look so low.
If every one knows us and we know none,
	'Tis time, I think, to trudge, pack, and be gone.
As from a bear a man would run for life,
	So fly I from her that would be my wife.
There's none but witches do inhabit here,
	And therefore 'tis high time that I were hence.
Master Antipholus!
Ay, that's my name.
I know it well, sir: lo, here is the chain.
I thought to have ta'en you at the Porpentine;
	The chain unfinish'd made me stay thus long.
What is your will that I shall do with this?
What please yourself, sir: I have made it for you.
Made it for me, sir!
I bespoke it not



	Not once, nor twice, but twenty times you have.
I pray you, sir, receive the money now,
	For fear you ne'er see chain nor money more.
You are a merry man, sir: fare you well.
I see, a man here needs not live by shifts,
	When in the streets he meets such golden gifts.
I'll to the mart, and there for Dromio stay:
	If any ship put out, then straight away.
Pleaseth you walk with me down to his house,
	I will discharge my bond, and thank you too.
That labour may you save: see where he comes.
I see the goldsmith.
Get thee gone;
	Buy thou a rope, and bring it home to me.
I buy a thousand pound a year: I buy a rope!
Belike you thought our love would last too long,
	If it were chain'd together, and therefore came not.
Saving your merry humour, here's the note
	How much your chain weighs to the utmost carat.
I am not furnish'd with the present money;
	Besides, I have some business in the town.
Then, you will bring the chain to her yourself?
No; bear it with you, lest I come not time enough.
Well, sir, I will.
Have you the chain about you?
An if I have not, sir, I hope you have,
	Or else you may return without your money.
Good Lord!
you use this dalliance to excuse
	Your breach of promise to the Porpentine.
I should have child you for not bringing it,
	But, like a shrew, you first begin to brawl.
The hour steals on; I pray you, sir, dispatch.
You hear how he importunes me: the chain!
Why, give it to my wife and fetch your money.
Come, come; you know I gave it you even now.
Either send the chain or send by me some token.
Fie!
now you run this humour out of breath.
Come, where's the chain?
I pray you, let me see it.
My business cannot brook this dalliance.
Good sir, say whe'r you'll answer me or no:
	If not, I'll leave him to the officer.
I answer you!
what should I answer you?
The money that you owe me for the chain.
I owe you none till I receive the chain.
You know I gave it you half an hour since.
You gave me none: you wrong me much to say so.
You wrong me more, sir, in denying it:
	Consider how it stands upon my credit.
Well, officer, arrest him at my suit.
I do;
	And charge you in the duke's name to obey me.
This touches me in reputation.
Either consent to pay this sum for me,
	Or I attach you by this officer.
Consent to pay thee that I never had!
Arrest me, foolish fellow, if thou dar'st.
Here is thy fee: arrest him, officer.
I would not spare my brother in this case,
	If he should scorn me so apparently.
I do arrest you, sir: you hear the suit.
I do obey thee till I give thee bail.
But, sirrah, you shall buy this sport as dear
	As all the metal in your shop will answer.
Sir, sir, I shall have law in Ephesus,
	To your notorious shame, I doubt it not.
Master, there is a bark of Epidamnum
	That stays but till her owner comes aboard,
	And then she bears away.
Our fraughtage, sir,
	I have convey'd aboard, and I have bought
	The oil, the balsamum, and aqua-vitæ.
Why, thou peevish sheep,
	What ship of Epidamnum stays for me?
A ship you sent me to, to hire waftage.
Thou drunken slave, I sent thee for a rope;
	And told thee to what purpose, and what end.
You sent me for a rope's end as soon:
	You sent me to the bay, sir, for a bark.
I will debate this matter at more leisure,
	And teach your ears to list me with more heed.
Tell her I am arrested in the street,
	And that shall bail me.
Hie thee, slave, be gone!
On, officer, to prison till it come.
that is where we din'd,
	Where Dowsabel did claim me for her husband:
	She is too big, I hope, for me to compass.
Thither I must, although against my will,
	For servants must their masters' minds fulfil.
Luciana, did he tempt thee so?
Mights thou perceive austerely in his eye
	That he did plead in earnest?
Look'd he or red or pale?
or sad or merrily?
What observation mad'st thou in this case
	Of his heart's meteors tilting in his face?
First he denied you had in him no right.
He meant he did me none; the more my spite.
Then swore he that he was a stranger here.
And true he swore, though yet forsworn he were.
Then pleaded I for you.
That love I begg'd for you he begg'd of me.
With what persuasion did he tempt thy love?
With words that in an honest suit might move.
First, he did praise my beauty, then my speech.
Didst speak him fair?
Have patience, I beseech.
I cannot, nor I will not hold me still:
	My tongue, though not my heart, shall have his will.
Who would be jealous then, of such a one?
No evil lost is wail'd when it is gone.
but I think him better than I say,
	And yet would herein others' eyes were worse.
Far from her nest the lapwing cries away:
	My heart prays for him, though my tongue do curse.
Here, go: the desk!
the purse!
sweet, now, make haste.
How hast thou lost thy breath?
By running fast.
Where is thy master, Dromio?
is he well?
No, he's in Tartar limbo, worse than hell.
Why, man, what is the matter?
I do not know the matter: he is 'rested on the case.
What, is he arrested?
tell me at whose suit.
I know not at whose suit he is arrested well;
	But he's in a suit of buff which 'rested him, that can I tell.
Will you send him, mistress, redemption, the money in his desk?
Not on a band, but on a stronger thing;
	A chain, a chain.
Do you not hear it ring?
What, the chain?
No, no, the bell: 'tis time that I were gone:
	It was two ere I left him, and now the clock strikes one.
The hours come back!
that did I never hear.
O yes; if any hour meet a sergeant, a' turns back for very fear.
As if Time were in debt!
how fondly dost thou reason!
Time is a very bankrupt, and owes more than he's worth to season.
Nay, he's a thief too: have you not heard men say,
	That Time comes stealing on by night and day?
If Time be in debt and theft, and a sergeant in the way,
	Hath he not reason to turn back an hour in a day?
Go, Dromio: there's the money, bear it straight,
	And bring thy master home immediately.
Come, sister; I am press'd down with conceit; Conceit, my comfort and my injury.
Sure these are but imaginary wiles,
	And Lapland sorcerers inhabit here.
Master, here's the gold you sent me for.
have you got the picture of old Adam new apparelled?
What gold is this?
What Adam dost thou mean?
I understand thee not.
What, thou meanest an officer?
Well, sir, there rest in your foolery.
Is there any ship puts forth to-night?
may we be gone?
Here are the angels that you sent for to deliver you.
The fellow is distract, and so am I;
	And here we wander in illusions:
	Some blessed power deliver us from hence!
Well met, well met, Master Antipholus.
I see, sir, you have found the goldsmith now:
	Is that the chain you promis'd me to-day?
Satan, avoid!
I charge thee tempt me not!
Master, is this Mistress Satan?
It is the devil.
Come not near her.
Your man and you are marvellous merry, sir.
Will you go with me?
we'll mend our dinner here.
Master, if you do, expect spoon-meat, so bespeak a long spoon.
Why, Dromio?
Marry, he must have a long spoon that must eat with the devil.
Avoid thee, fiend!
what tell'st thou me of supping?
Thou art, as you are all, a sorceress:
	I conjure thee to leave me and be gone.
Master, be wise: an if you give it her,
	The devil will shake her chain and fright us with it.
I pray you, sir, my ring, or else the chain:
	I hope you do not mean to cheat me so.
Avaunt, thou witch!
Come, Dromio, let us go.
'Fly pride,' says the peacock: mistress, that you know.
Now, out of doubt, Antipholus is mad,
	Else would he never so demean himself.
This course I fittest choose,
	For forty ducats is too much to lose.
My wife is in a wayward mood to-day,
	And will not lightly trust the messenger.
That I should be attach'd in Ephesus,
	I tell you, 'twill sound harshly in her ears.
Here comes my man: I think he brings the money.
have you that I sent you for?
Here's that, I warrant you, will pay them all.
But where's the money?
Why, sir, I gave the money for the rope.
Five hundred ducats, villain, for a rope?
I'll serve you, sir, five hundred at the rate.
To what end did I bid thee hie thee home?
To a rope's end, sir; and to that end am I return'd.
And to that end, sir, I will welcome you.
Good sir, be patient.
Nay, 'tis for me to be patient; I am in adversity.
Good now, hold thy tongue.
Nay, rather persuade him to hold his hands.
Thou whoreson, senseless villain!
I would I were senseless, sir, that I might not feel your blows.
Thou art sensible in nothing but blows, and so is an ass.
I am an ass indeed; you may prove it by my long ears.
I have served him from the hour of my nativity to this instant, and have nothing at his hands for my service but blows.
Come, go along; my wife is coming yonder.
Mistress, respice finem, respect your end; or rather, to prophesy like the parrot, 'Beware the rope's end.'
Wilt thou still talk?
How say you now?
is not your husband mad?
His incivility confirms no less.
how fiery and how sharp he looks.
Mark how he trembles in his ecstasy!
Give me your hand and let me feel your pulse.
There is my hand, and let it feel your ear.
Peace, doting wizard, peace!
I am not mad.
that thou wert not, poor distressed soul!
You minion, you, are these your customers?
O husband, God doth know you din'd at home;
	Where would you had remain'd until this time.
Free from these slanders and this open shame!
Din'd at home!
Thou villain, what say'st thou?
Sir, sooth to say, you did not dine at home.
Were not my doors lock'd up and I shut out?
Perdy, your doors were lock'd and you shut out.
Sans fable, she herself revil'd you there.
Did not her kitchen-maid rail, taunt, and scorn me?
Certes, she did; the kitchen-vestal scorn'd you.
And did not I in rage depart from thence?
In verity you did: my bones bear witness,
	That since have felt the vigour of his rage.
Is't good to soothe him in these contraries?
It is no shame: the fellow finds his vein,
	And, yielding to him humours well his frenzy.
Thou hast suborn'd the goldsmith to arrest me.
I sent you money to redeem you,
	By Dromio here, who came in haste for it.
Money by me!
heart and good will you might;
	But surely, master, not a rag of money.
Went'st not thou to her for a purse of ducats?
He came to me, and I deliver'd it.
And I am witness with her that she did.
God and the rope-maker bear me witness
	That I was sent for nothing but a rope!
Mistress, both man and master is possess'd:
	I know it by their pale and deadly looks.
They must be bound and laid in some dark room.
Say, wherefore didst thou lock me forth to-day?
And why dost thou deny the bag of gold?
I did not, gentle husband, lock thee forth.
And, gentle master, I receiv'd no gold;
	But I confess, sir, that we were lock'd out.
Dissembling villain!
thou speak'st false in both.
Dissembling harlot!
bind him, bind him, let him not come near me.
More company!
the fiend is strong within him.
Ay me!
poor man, how pale and wan he looks!
Thou gaoler, thou,
	I am thy prisoner: wilt thou suffer them
	To make a rescue?
Masters, let him go:
	He is my prisoner, and you shall not have him.
Go bind this man, for he is frantic too.
What wilt thou do, thou peevish officer?
Hast thou delight to see a wretched man
	Do outrage and displeasure to himself?
He is my prisoner: if I let him go,
	The debt he owes will be requir'd of me.
Good Master doctor, see him safe convey'd
	Home to my house.
O most unhappy day!
O most unhappy strumpet!
Master, I am here enter'd in bond for you.
Out on thee, villain!
wherefore dost thou mad me?
Will you be bound for nothing?
be mad, good master; cry, 'the devil!'
God help, poor souls!
how idly do they talk.
Go bear him hence.
Sister, go you with me.—



	Say now, whose suit is he arrested at?
One Angelo, a goldsmith; do you know him?
I know the man.
What is the sum he owes?
Two hundred ducats.
Say, how grows it due?
Due for a chain your husband had of him.
He did bespeak a chain for me, but had it not.
It may be so, but I did never see it.
Come, gaoler, bring me where the goldsmith is:
	I long to know the truth hereof at large.
God, for thy mercy!
And come with naked swords.
Let's call more help
	To have them bound again.
Away!
they'll kill us.
I see, these witches are afraid of swords.
She that would be your wife now ran from you.
Come to the Centaur; fetch our stuff from thence:
	I long that we were safe and sound aboard.
I will not stay to-night for all the town;
	Therefore away, to get our stuff aboard.
How is the man esteem'd here in the city?
Speak softly: yonder, as I think, he walks.
'Tis so; and that self chain about his neck
	Which he forswore most monstrously to have.
Good sir, draw near to me, I'll speak to him.
This chain you had of me; can you deny it?
I think I had: I never did deny it.
Yes, that you did, sir, and forswore it too.
Who heard me to deny it or forswear it?
These ears of mine, thou know'st, did hear thee.
Fie on thee, wretch!
'tis pity that thou liv'st
	To walk where any honest men resort.
I dare, and do defy thee for a villain.
Hold!
hurt him not, for God's sake!
he is mad.
Some get within him, take his sword away.
Bind Dromio too, and bear them to my house.
Run, master, run; for God's sake, take a house!
Be quiet, people.
Wherefore throng you hither?
To fetch my poor distracted husband hence.
Let us come in, that we may bind him fast,
	And bear him home for his recovery.
I knew he was not in his perfect wits.
I am sorry now that I did draw on him.
How long hath this possession held the man?
Hath he not lost much wealth by wrack of sea?
Buried some dear friend?
Hath not else his eye
	Stray'd his affection in unlawful love?
A sin prevailing much in youthful men,
	Who give their eyes the liberty of gazing.
Which of these sorrows is he subject to?
To none of these, except it be the last;
	Namely, some love that drew him oft from home.
Ay, but not rough enough.
As roughly as my modesty would let me.
Ay, but not enough.
It seems, his sleeps were hinder'd by thy railing,
	And thereof comes it that his head is light.
She never reprehended him but mildly
	When he demean'd himself rough, rude, and wildly.
Why bear you these rebukes and answer not?
Good people, enter, and lay hold on him.
No; not a creature enters in my house.
Then, let your servants bring my husband forth.
It is a branch and parcel of mine oath,
	A charitable duty of my order;
	Therefore depart and leave him here with me.
Be quiet, and depart: thou shalt not have him.
Complain unto the duke of this indignity.
Upon what cause?
See where they come: we will behold his death.
Kneel to the duke before he pass the abbey.
Yet once again proclaim it publicly,
	If any friend will pay the sum for him,
	He shall not die; so much we tender him.
Justice, most sacred duke, against the abbess!
She is a virtuous and a reverend lady:
	It cannot be that she hath done thee wrong.
Therefore, most gracious duke, with thy command
	Let him be brought forth, and borne hence for help.
Go, some of you, knock at the abbey gate
	And bid the lady abbess come to me.
I will determine this before I stir.
O mistress, mistress!
shift and save yourself!
Peace, fool!
thy master and his man are here,
	And that is false thou dost report to us.
Mistress, upon my life, I tell you true;
	I have not breath'd almost, since I did see it.
He cries for you and vows, if he can take you,
	To scotch your face, and to disfigure you.
Hark, hark!
I hear him, mistress: fly, be gone!
Come, stand by me; fear nothing.
Ay me, it is my husband!
Justice, most gracious duke!
Unless the fear of death doth make me dote,
	I see my son Antipholus and Dromio!
Justice, sweet prince, against that woman there!
Beyond imagination is the wrong
	That she this day hath shameless thrown on me.
Discover how, and thou shalt find me just.
This day, great duke, she shut the doors upon me,
	While she with harlots feasted in my house.
A grievous fault!
Say, woman, didst thou so?
No, my good lord: myself, he, and my sister
	To-day did dine together.
So befall my soul
	As this is false he burdens me withal!
Ne'er may I look on day, nor sleep on night,
	But she tells to your highness simple truth!
O perjur'd woman!
They are both forsworn:
	In this the madman justly chargeth them!
I did obey, and sent my peasant home
	For certain ducats: he with none return'd.
Then fairly I bespoke the officer
	To go in person with me to my house.
My lord, in truth, thus far I witness with him,
	That he din'd not at home, but was lock'd out.
But had he such a chain of thee, or no?
He had, my lord; and when he ran in here,
	These people saw the chain about his neck.
And this is false you burden me withal.
I think you all have drunk of Circe's cup.
Sirrah, what say you?
Sir, he din'd with her there, at the Porpentine.
He did, and from my finger snatch'd that ring.
'Tis true, my liege; this ring I had of her.
Saw'st thou him enter at the abbey here?
As sure, my liege, as I do see your Grace.
Why, this is strange.
Go call the abbess hither.
I think you are all mated or stark mad.
Speak freely, Syracusian, what thou wilt.
Is not your name, sir, called Antipholus?
And is not that your bondman Dromio?
I am sure you both of you remember me.
Ourselves we do remember, sir, by you;
	For lately we were bound, as you are now.
You are not Pinch's patient, are you, sir?
Why look you strange on me?
you know me well.
I never saw you in my life till now.
Neither.
Dromio, nor thou?
No, trust me, sir, not I.



	I am sure thou dost.
Ay, sir; but I am sure I do not; and whatsoever a man denies, you are now bound to believe him.
Not know my voice!
I never saw my father in my life.
The duke and all that know me in the city
	Can witness with me that it is not so:
	I ne'er saw Syracusa in my life.
I tell thee, Syracusian, twenty years
	Have I been patron to Antipholus,
	During which time he ne'er saw Syracusa.
I see thy age and dangers make thee dote.
Most mighty duke, behold a man much wrong'd.
I see two husbands, or mine eyes deceive me!
One of these men is Genius to the other;
	And so of these: which is the natural man,
	And which the spirit?
I, sir, am Dromio: command him away.
I, sir, am Dromio: pray let me stay.
Ægeon art thou not?
or else his ghost?
my old master; who hath bound him here?
Whoever bound him, I will loose his bonds,
	And gain a husband by his liberty.
if thou be'st the same Ægeon, speak,
	And speak unto the same Æmilia!
If I dream not, thou art Æmilia:
	If thou art she, tell me where is that son
	That floated with thee on the fatal raft?
What then became of them, I cannot tell;
	I to this fortune that you see me in.
Antipholus, thou cam'st from Corinth first?
No, sir, not I; I came from Syracuse.
Stay, stand apart; I know not which is which.
I came from Corinth, my most gracious lord,—



	And I with him.
Brought to this town by that most famous warrior,
	Duke Menaphon, your most renowned uncle.
Which of you two did dine with me to-day?
I, gentle mistress.
And are not you my husband?
No; I say nay to that.
And so do I; yet did she call me so;
	And this fair gentlewoman, her sister here,
	Did call me brother.
What I told you then,
	I hope I shall have leisure to make good,
	If this be not a dream I see and hear.
That is the chain, sir, which you had of me.
I think it be, sir; I deny it not.
And you, sir, for this chain arrested me.
I think I did, sir; I deny it not.
I sent you money, sir, to be your bail,
	By Dromio; but I think he brought it not.
No, none by me.
This purse of ducats I receiv'd from you,
	And Dromio, my man, did bring them me.
These ducats pawn I for my father here.
It shall not need: thy father hath his life.
Sir, I must have that diamond from you.
There, take it; and much thanks for my good cheer.
With all my heart I'll gossip at this feast.
Master, shall I fetch your stuff from shipboard?
Dromio, what stuff of mine hast thou embark'd?
Your goods that lay at host, sir, in the Centaur.
He speaks to me.
I am your master, Dromio:
	Come, go with us; we'll look to that anon:
	Embrace thy brother there; rejoice with him.
Methinks you are my glass, and not my brother:
	I see by you I am a sweet-fac'd youth.
Will you walk in to see their gossiping?
Not I, sir; you are my elder.
That's a question: how shall we try it?
We'll draw cuts for the senior: till then lead thou first.

"""
meteor_scores, bert_scores, rouge_scores = evaluate_summaries(generated_summary, reference_summary)

# Print evaluation scores
print("METEOR scores:")
for method, score in meteor_scores.items():
    print(f"{method}: {score}")
print()

print("BERT scores:")
for method, score in bert_scores.items():
    print(f"{method}: {score}")
print()

print("ROUGE scores:")
for method, score in rouge_scores.items():
    print(f"{method}: {score}")



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


METEOR scores:
Generated: 0.1648331601599218

BERT scores:
Generated: 0.7918664216995239

ROUGE scores:
Generated: {'rouge-1': {'r': 0.4354243542435424, 'p': 0.054528650646950096, 'f': 0.0969199158863292}, 'rouge-2': {'r': 0.1016260162601626, 'p': 0.00792393026941363, 'f': 0.014701557023190892}, 'rouge-l': {'r': 0.42435424354243545, 'p': 0.05314232902033272, 'f': 0.0944558501779103}}


In [ ]:
#3
#LSA
generated_summary="""Thy substance, valu'd at the highest rate, Cannot amount unto a hundred marks; Therefore, by law thou art condemn'd to die. She that doth call me husband, even my soul Doth for a wife abhor; but her fair sister, Possess'd with such a gentle sovereign grace, Of such enchanting presence and discourse, Hath almost made me traitor to myself: But, lest myself be guilty to self-wrong, I'll stop mine ears against the mermaid's song. May it please your Grace, Antipholus, my husband, Whom I made lord of me and all I had, At your important letters, this ill day A most outrageous fit of madness took him, That desperately he hurried through the street,— With him his bondman, all as mad as he,— Doing displeasure to the citizens By rushing in their houses, bearing thence Rings, jewels, anything his rage did like."""
meteor_scores, bert_scores, rouge_scores = evaluate_summaries(generated_summary, reference_summary)

# Print evaluation scores
print("METEOR scores:")
for method, score in meteor_scores.items():
    print(f"{method}: {score}")
print()

print("BERT scores:")
for method, score in bert_scores.items():
    print(f"{method}: {score}")
print()

print("ROUGE scores:")
for method, score in rouge_scores.items():
    print(f"{method}: {score}")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


METEOR scores:
Generated: 0.08212309820193638

BERT scores:
Generated: 0.7828030586242676

ROUGE scores:
Generated: {'rouge-1': {'r': 0.1033210332103321, 'p': 0.23140495867768596, 'f': 0.14285713858925983}, 'rouge-2': {'r': 0.006097560975609756, 'p': 0.02127659574468085, 'f': 0.009478669523147644}, 'rouge-l': {'r': 0.0959409594095941, 'p': 0.21487603305785125, 'f': 0.13265305695660679}}


In [ ]:
#4
#bert extractive
generated_summary="""Proceed, Solinus, to procure my fall,
	And by the doom of death end woes and all. That very hour, and in the self-same inn,
	A meaner woman was delivered
	Of such a burden, male twins, both alike. My wife, not meanly proud of two such boys,
	Made daily motions for our home return:
	Unwilling I agreed; alas! I from my mistress come to you in post;
	If I return, I shall be post indeed,
	For she will score your fault upon my pate. Methinks your maw, like mine, should be your clock
	And strike you home without a messenger. I have some marks of yours upon my pate,
	Some of my mistress' marks upon my shoulders,
	But not a thousand marks between you both. Upon my life, by some device or other
	The villain is o'er-raught of all my money. no marvel though she pause;
	They can be meek that have no other cause. Spake he so doubtfully, thou couldst not feel his meaning? When I desir'd him to come home to dinner,
	He ask'd me for a thousand marks in gold:
	''Tis dinner time,' quoth I; 'my gold!' quoth he:
	'Where is the thousand marks I gave thee, villain?' Hath homely age the alluring beauty took
	From my poor cheek? Wast thou mad,

	That thus so madly thou didst answer me? Marry, sir, for this something that you gave me for nothing. No, sir: I think the meat wants that I have



	In good time, sir; what's that? There's no time for a man to recover his hair that grows bald by nature. Because it is a blessing that he bestows on beasts: and what he hath scanted men in hair, he hath given them in wit. Not a man of those but he hath the wit to lose his hair. The one, to save the money that he spends in tiring; the other, that at dinner they should not drop in his porridge. Thus I mend it: Time himself is bald, and therefore to the world's end will have bald followers. how comes it,
	That thou art thus estranged from thyself? Thyself I call it, being strange to me,
	That, undividable, incorporate,
	Am better than thy dear self's better part. Dromio, thou drone, thou snail, thou slug, thou sot! Sirrah, if any ask you for your master,
	Say he dines forth, and let no creature enter. Good meat, sir, is common; that every churl affords. And welcome more common, for that's nothing but words. The porter for this time, sir, and my name is Dromio. Here is neither cheer, sir, nor welcome: we would fain have either. They stand at the door, master: bid them welcome hither. Against my soul's pure truth why labour you
	To make it wander in an unknown field? I am an ass, I am a woman's man and besides myself. Swart, like my shoe, but her face nothing like so clean kept: for why she sweats; a man may go over shoes in the grime of it. No, sir, 'tis in grain; Noah's flood could not do it. Marry, sir, in her buttocks: I found it out by the bogs. In her forehead; armed and reverted, making war against her heir. Go home with it and please your wife withal;
	And soon at supper-time I'll visit you,
	And then receive my money for the chain. I see, a man here needs not live by shifts,
	When in the streets he meets such golden gifts. Saving your merry humour, here's the note
	How much your chain weighs to the utmost carat. An if I have not, sir, I hope you have,
	Or else you may return without your money. You wrong me more, sir, in denying it:
	Consider how it stands upon my credit. I would not spare my brother in this case,
	If he should scorn me so apparently. Thou drunken slave, I sent thee for a rope;
	And told thee to what purpose, and what end. Thither I must, although against my will,
	For servants must their masters' minds fulfil. but I think him better than I say,
	And yet would herein others' eyes were worse. I know not at whose suit he is arrested well;
	But he's in a suit of buff which 'rested him, that can I tell. This I wonder at:
	That he, unknown to me, should be in debt:
	Tell me, was he arrested on a band? Nay, he's a thief too: have you not heard men say,
	That Time comes stealing on by night and day? have you got the picture of old Adam new apparelled? A ring he hath of mine worth forty ducats,
	And for the same he promis'd me a chain:
	Both one and other he denies me now. Thou art sensible in nothing but blows, and so is an ass. I am an ass indeed; you may prove it by my long ears. Good Doctor Pinch, you are a conjurer;
	Establish him in his true sense again,
	And I will please you what you will demand. Thou gaoler, thou,
	I am thy prisoner: wilt thou suffer them
	To make a rescue? I will discharge thee ere I go from thee:
	Bear me forthwith unto his creditor,
	And, knowing how the debt grows, I will pay it. Good Master doctor, see him safe convey'd
	Home to my house. Hath not else his eye
	Stray'd his affection in unlawful love? She never reprehended him but mildly
	When he demean'd himself rough, rude, and wildly. Beyond imagination is the wrong
	That she this day hath shameless thrown on me. So befall my soul
	As this is false he burdens me withal! Within this hour I was his bondman, sir;
	But he, I thank him, gnaw'd in two my cords:
	Now am I Dromio and his man, unbound. But seven years since, in Syracusa, boy,
	Thou know'st we parted: but perhaps, my son,
	Thou sham'st to acknowledge me in misery. Brought to this town by that most famous warrior,
	Duke Menaphon, your most renowned uncle. I sent you money, sir, to be your bail,
	By Dromio; but I think he brought it not. Thirty-three years have I but gone in travail
	Of you, my sons; and, till this present hour
	My heavy burdens ne'er delivered. I am your master, Dromio:
	Come, go with us; we'll look to that anon:
	Embrace thy brother there; rejoice with him. There is a fat friend at your master's house,
	That kitchen'd me for you to-day at dinner:
	She now shall be my sister, not my wife. We'll draw cuts for the senior: till then lead thou first.

"""
meteor_scores, bert_scores, rouge_scores = evaluate_summaries(generated_summary, reference_summary)

# Print evaluation scores
print("METEOR scores:")
for method, score in meteor_scores.items():
    print(f"{method}: {score}")
print()

print("BERT scores:")
for method, score in bert_scores.items():
    print(f"{method}: {score}")
print()

print("ROUGE scores:")
for method, score in rouge_scores.items():
    print(f"{method}: {score}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


METEOR scores:
Generated: 0.25487613824202443

BERT scores:
Generated: 0.780019998550415

ROUGE scores:
Generated: {'rouge-1': {'r': 0.2656826568265683, 'p': 0.12392426850258176, 'f': 0.16901408016897562}, 'rouge-2': {'r': 0.03048780487804878, 'p': 0.014285714285714285, 'f': 0.019455248573030087}, 'rouge-l': {'r': 0.23616236162361623, 'p': 0.11015490533562823, 'f': 0.15023473744597093}}


In [ ]:
#5
#BART ABSTRACTIVE
generated_summary="""Solinus, a merchant of Syracusa, tells the story of his death. He describes how his wife and two sons died in a shipwreck. Solinus describes his wife's weeping as she mourned her husband's death. The story ends with Solinus' death at the hands of his wife."""
meteor_scores, bert_scores, rouge_scores = evaluate_summaries(generated_summary, reference_summary)

# Print evaluation scores
print("METEOR scores:")
for method, score in meteor_scores.items():
    print(f"{method}: {score}")
print()

print("BERT scores:")
for method, score in bert_scores.items():
    print(f"{method}: {score}")
print()

print("ROUGE scores:")
for method, score in rouge_scores.items():
    print(f"{method}: {score}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


METEOR scores:
Generated: 0.042329412893406776

BERT scores:
Generated: 0.8312814235687256

ROUGE scores:
Generated: {'rouge-1': {'r': 0.08118081180811808, 'p': 0.6470588235294118, 'f': 0.14426229310099437}, 'rouge-2': {'r': 0.014227642276422764, 'p': 0.16279069767441862, 'f': 0.026168222820787925}, 'rouge-l': {'r': 0.08118081180811808, 'p': 0.6470588235294118, 'f': 0.14426229310099437}}


In [ ]:
#6
#T5 ABSTRACTIVE
generated_summary="""eric liu: if any, thou departedst from the bay of Ephesus, he dies. liu: if any, thou departedst from thy native home, And for what cause thou cam'st to Ephesus. he says the world may witness that my end Was wrought by nature, not by vile offence. liu: thou liv'd in
"""
meteor_scores, bert_scores, rouge_scores = evaluate_summaries(generated_summary, reference_summary)

# Print evaluation scores
print("METEOR scores:")
for method, score in meteor_scores.items():
    print(f"{method}: {score}")
print()

print("BERT scores:")
for method, score in bert_scores.items():
    print(f"{method}: {score}")
print()

print("ROUGE scores:")
for method, score in rouge_scores.items():
    print(f"{method}: {score}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


METEOR scores:
Generated: 0.0291005291005291

BERT scores:
Generated: 0.7800291776657104

ROUGE scores:
Generated: {'rouge-1': {'r': 0.04797047970479705, 'p': 0.3333333333333333, 'f': 0.08387096554235177}, 'rouge-2': {'r': 0.0040650406504065045, 'p': 0.044444444444444446, 'f': 0.007448788036162318}, 'rouge-l': {'r': 0.04797047970479705, 'p': 0.3333333333333333, 'f': 0.08387096554235177}}


In [ ]:
#7
#BERT ABSTRACTIVE
generated_summary="""Thou art, as you are all, a sorceress:
	I conjure thee to leave me and be gone. On, officer, to prison till it come.
"""
meteor_scores, bert_scores, rouge_scores = evaluate_summaries(generated_summary, reference_summary)

# Print evaluation scores
print("METEOR scores:")
for method, score in meteor_scores.items():
    print(f"{method}: {score}")
print()

print("BERT scores:")
for method, score in bert_scores.items():
    print(f"{method}: {score}")
print()

print("ROUGE scores:")
for method, score in rouge_scores.items():
    print(f"{method}: {score}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


METEOR scores:
Generated: 0.0177367860943597

BERT scores:
Generated: 0.779609739780426

ROUGE scores:
Generated: {'rouge-1': {'r': 0.02952029520295203, 'p': 0.3333333333333333, 'f': 0.05423728664085037}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.02952029520295203, 'p': 0.3333333333333333, 'f': 0.05423728664085037}}


In [ ]:
#8
#FREQUENCY BASED
generated_summary="""
	Proceed, Solinus, to procure my fall,
	And by the doom of death end woes and all. Merchant of Syracusa, plead no more. I am not partial to infringe our laws:
	The enmity and discord which of late
	Sprung from the rancorous outrage of your duke
	To merchants, our well-dealing countrymen,
	Who, wanting guilders to redeem their lives,
	Have seal'd his rigorous statutes with their bloods,
	Excludes all pity from our threat'ning looks. For, since the mortal and intestine jars
	'Twixt thy seditious countrymen and us,
	It hath in solemn synods been decreed,
	Both by the Syracusians and ourselves,
	T' admit no traffic to our adverse towns:
	Nay, more, if any, born at Ephesus
	Be seen at Syracusian marts and fairs;
	Again, if any Syracusian born
	Come to the bay of Ephesus, he dies,
	His goods confiscate to the duke's dispose;
	Unless a thousand marks be levied,
	To quit the penalty and to ransom him. Thy substance, valu'd at the highest rate,
	Cannot amount unto a hundred marks;
	Therefore, by law thou art condemn'd to die. Yet this my comfort: when your words are done,
	My woes end likewise with the evening sun. Well, Syracusian; say, in brief the cause
	Why thou departedst from thy native home,
	And for what cause thou cam'st to Ephesus. A heavier task could not have been impos'd
	Than I to speak my griefs unspeakable;
	Yet, that the world may witness that my end
	Was wrought by nature, not by vile offence,
	I'll utter what my sorrow gives me leave. In Syracusa was I born, and wed
	Unto a woman, happy but for me,
	And by me too, had not our hap been bad. With her I liv'd in joy: our wealth increas'd
	By prosperous voyages I often made
	To Epidamnum; till my factor's death,
	And the great care of goods at random left,
	Drew me from kind embracements of my spouse:
	From whom my absence was not six months old,
	Before herself,—almost at fainting under
	The pleasing punishment that women bear,—
	Had made provision for her following me,
	And soon and safe arrived where I was. There had she not been long but she became
	A joyful mother of two goodly sons;
	And, which was strange, the one so like the other,
	As could not be distinguish'd but by names. That very hour, and in the self-same inn,
	A meaner woman was delivered
	Of such a burden, male twins, both alike. Those,—for their parents were exceeding poor,—
	I bought, and brought up to attend my sons. My wife, not meanly proud of two such boys,
	Made daily motions for our home return:
	Unwilling I agreed; alas! A league from Epidamnum had we sail'd,
	Before the always-wind-obeying deep
	Gave any tragic instance of our harm:
	But longer did we not retain much hope;
	For what obscured light the heavens did grant
	Did but convey unto our fearful minds
	A doubtful warrant of immediate death;
	Which, though myself would gladly have embrac'd,
	Yet the incessant weepings of my wife,
	Weeping before for what she saw must come,
	And piteous plainings of the pretty babes,
	That mourn'd for fashion, ignorant what to fear,
	Forc'd me to seek delays for them and me. And this it was, for other means was none:
	The sailors sought for safety by our boat,
	And left the ship, then sinking-ripe, to us:
	My wife, more careful for the latter-born,
	Had fasten'd him unto a small spare mast,
	Such as seafaring men provide for storms;
	To him one of the other twins was bound,
	Whilst I had been like heedful of the other. The children thus dispos'd, my wife and I,
	Fixing our eyes on whom our care was fix'd,
	Fasten'd ourselves at either end the mast;
	And floating straight, obedient to the stream,
	Were carried towards Corinth, as we thought. At length the sun, gazing upon the earth,
	Dispers'd those vapours that offended us,
	And, by the benefit of his wished light
	The seas wax'd calm, and we discovered
	Two ships from far making amain to us;
	Of Corinth that, of Epidaurus this:
	But ere they came,—O! Nay, forward, old man; do not break off so;
	For we may pity, though not pardon thee. For, ere the ships could meet by twice five leagues,
	We were encounter'd by a mighty rock;
	Which being violently borne upon,
	Our helpful ship was splitted in the midst;
	So that, in this unjust divorce of us
	Fortune had left to both of us alike
	What to delight in, what to sorrow for. seeming as burdened
	With lesser weight, but not with lesser woe,
	Was carried with more speed before the wind,
	And in our sight they three were taken up
	By fishermen of Corinth, as we thought. At length, another ship had seiz'd on us;
	And, knowing whom it was their hap to save,
	Gave healthful welcome to their ship-wrack'd guests;
	And would have reft the fishers of their prey,
	Had not their bark been very slow of sail;
	And therefore homeward did they bend their course. Thus have you heard me sever'd from my bliss,
	That by misfortune was my life prolong'd,
	To tell sad stories of my own mishaps. And, for the sake of them thou sorrowest for,
	Do me the favour to dilate at full
	What hath befall'n of them and thee till now. My youngest boy, and yet my eldest care,
	At eighteen years became inquisitive
	After his brother; and importun'd me
	That his attendant—for his case was like,
	Reft of his brother, but retain'd his name—
	Might bear him company in the quest of him;
	Whom whilst I labour'd of a love to see,
	I hazarded the loss of whom I lov'd. Five summers have I spent in furthest Greece,
	Roaming clean through the bounds of Asia,
	And, coasting homeward, came to Ephesus,
	Hopeless to find, yet loath to leave unsought
	Or that or any place that harbours men. But here must end the story of my life;
	And happy were I in my timely death,
	Could all my travels warrant me they live. Now, trust me, were it not against our laws,
	Against my crown, my oath, my dignity,
	Which princes, would they, may not disannul,
	My soul should sue as advocate for thee. But though thou art adjudged to the death
	And passed sentence may not be recall'd
	But to our honour's great disparagement,
	Yet will I favour thee in what I can:
	Therefore, merchant, I'll limit thee this day
	To seek thy life by beneficial help. Try all the friends thou hast in Ephesus;
	Beg thou, or borrow, to make up the sum,
	And live; if no, then thou art doom'd to die. Gaoler, take him to thy custody. I will, my lord. Hopeless and helpless doth Ægeon wend,
	But to procrastinate his lifeless end. Therefore, give out you are of Epidamnum,
	Lest that your goods too soon be confiscate. This very day, a Syracusian merchant
	Is apprehended for arrival here;
	And, not being able to buy out his life,
	According to the statute of the town
	Dies ere the weary sun set in the west. Go bear it to the Centaur, where we host,
	And stay there, Dromio, till I come to thee. Within this hour it will be dinner-time:
	Till that, I'll view the manners of the town,
	Peruse the traders, gaze upon the buildings,
	And then return and sleep within mine inn,
	For with long travel I am stiff and weary. Many a man would take you at your word,
	And go indeed, having so good a mean. A trusty villain, sir, that very oft,
	When I am dull with care and melancholy,
	Lightens my humour with his merry jests. I am invited, sir, to certain merchants,
	Of whom I hope to make much benefit;
	I crave your pardon. Soon at five o'clock,
	Please you, I'll meet with you upon the mart,
	And afterward consort you till bed-time:
	My present business calls me from you now. Farewell till then: I will go lose myself,
	And wander up and down to view the city. Sir, I commend you to your own content. He that commends me to mine own content,
	Commends me to the thing I cannot get. I to the world am like a drop of water
	That in the ocean seeks another drop;
	Who, falling there to find his fellow forth,
	Unseen, inquisitive, confounds himself:
	So I, to find a mother and a brother,
	In quest of them, unhappy, lose myself. rather approach'd too late:
	The capon burns, the pig falls from the spit,
	The clock hath strucken twelve upon the bell;
	My mistress made it one upon my cheek:
	She is so hot because the meat is cold;
	The meat is cold because you come not home;
	You come not home because you have no stomach;
	You have no stomach, having broke your fast;
	But we, that know what 'tis to fast and pray,
	Are penitent for your default to-day. Stop in your wind, sir: tell me this, I pray:
	Where have you left the money that I gave you? O!—sixpence, that I had o' Wednesday last
	To pay the saddler for my mistress' crupper;
	The saddler had it, sir; I kept it not. I pray you, jest, sir, as you sit at dinner. I from my mistress come to you in post;
	If I return, I shall be post indeed,
	For she will score your fault upon my pate. Methinks your maw, like mine, should be your clock
	And strike you home without a messenger. Come, Dromio, come; these jests are out of season;
	Reserve them till a merrier hour than this. why, you gave no gold to me. Come on, sir knave, have done your foolishness,
	And tell me how thou hast dispos'd thy charge. My charge was but to fetch you from the mart
	Home to your house, the Phœnix, sir, to dinner:
	My mistress and her sister stays for you. Now, as I am a Christian, answer me,
	In what safe place you have bestow'd my money;
	Or I shall break that merry sconce of yours
	That stands on tricks when I am undispos'd. I have some marks of yours upon my pate,
	Some of my mistress' marks upon my shoulders,
	But not a thousand marks between you both. If I should pay your worship those again,
	Perchance you will not bear them patiently. Your worship's wife, my mistress at the Phœnix;
	She that doth fast till you come home to dinner,
	And prays that you will hie you home to dinner. There, take you that, sir knave. Nay, an you will not, sir, I'll take my heels. Upon my life, by some device or other
	The villain is o'er-raught of all my money. They say this town is full of cozenage;
	As, nimble jugglers that deceive the eye,
	Dark-working sorcerers that change the mind,
	Soul-killing witches that deform the body,
	Disguised cheaters, prating mountebanks,
	And many such-like liberties of sin:
	If it prove so, I will be gone the sooner. I'll to the Centaur, to go seek this slave:
	I greatly fear my money is not safe. Sure, Luciana, it is two o'clock. Perhaps some merchant hath invited him,
	And from the mart he's somewhere gone to dinner. Good sister, let us dine and never fret:
	A man is master of his liberty:
	Time is their master, and, when they see time,
	They'll go or come: if so, be patient, sister. Look, when I serve him so, he takes it ill.



	O! Why, headstrong liberty is lash'd with woe. There's nothing situate under heaven's eye
	But hath his bound, in earth, in sea, in sky:
	The beasts, the fishes, and the winged fowls,
	Are their males' subjects and at their controls. Men, more divine, the masters of all these,
	Lords of the wide world, and wild wat'ry seas,
	Indu'd with intellectual sense and souls,
	Of more pre-eminence than fish and fowls,
	Are masters to their females and their lords:
	Then, let your will attend on their accords. Not this, but troubles of the marriage-bed. But, were you wedded, you would bear some sway. Ere I learn love, I'll practise to obey. Till he come home again, I would forbear. A wretched soul, bruis'd with adversity,
	We bid be quiet when we hear it cry;
	But were we burden'd with like weight of pain,
	As much, or more we should ourselves complain:
	So thou, that hast no unkind mate to grieve thee,
	With urging helpless patience wouldst relieve me:
	But if thou live to see like right bereft. Well, I will marry one day, but to try. Nay, he's at two hands with me, and that my two ears can witness. Ay, ay, he told his mind upon mine ear. Beshrew his hand, I scarce could understand it. Nay, he struck so plainly, I could too well feel his blows; and withal so doubtfully, that I could scarce understand them. Why, mistress, sure my master is horn-mad. I mean not cuckold-mad; but, sure, he is stark mad. When I desir'd him to come home to dinner,
	He ask'd me for a thousand marks in gold:
	''Tis dinner time,' quoth I; 'my gold!' quoth he:
	'Your meat doth burn,' quoth I; 'my gold!' quoth he:
	'Where is the thousand marks I gave thee, villain?' 'The pig,' quoth I, 'is burn'd;' 'my gold!' quoth he:
	'My mistress, sir,' quoth I: 'hang up thy mistress! Quoth my master:
	'I know,' quoth he, 'no house, no wife, no mistress.' So that my errand, due unto my tongue,
	I thank him, I bear home upon my shoulders;
	For, in conclusion, he did beat me there. Go back again, thou slave, and fetch him home. For God's sake, send some other messenger. Back, slave, or I will break thy pate across. And he will bless that cross with other beating:
	Between you, I shall have a holy head. You spurn me hence, and he will spurn me hither:
	If I last in this service, you must case me in leather. His company must do his minions grace,
	Whilst I at home starve for a merry look. If voluble and sharp discourse be marr'd,
	Unkindness blunts it more than marble hard:
	Do their gay vestments his affections bait? My decayed fair
	A sunny look of his would soon repair;
	But, too unruly deer, he breaks the pale
	And feeds from home: poor I am but his stale. Sister, you know he promis'd me a chain:
	Would that alone, alone he would detain,
	So he would keep fair quarter with his bed! I see, the jewel best enamelled
	Will lose his beauty; and though gold bides still
	That others touch, yet often touching will
	Wear gold; and no man that hath a name,
	By falsehood and corruption doth it shame. Since that my beauty cannot please his eye,
	I'll weep what's left away, and weeping die. The gold I gave to Dromio is laid up
	Safe at the Centaur; and the heedful slave
	Is wander'd forth, in care to seek me out. By computation, and mine host's report,
	I could not speak with Dromio since at first
	I sent him from the mart. See, here he comes. How now, sir! As you love strokes, so jest with me again. Even now, even here, not half-an-hour since. I did not see you since you sent me hence,
	Home to the Centaur, with the gold you gave me. Villain, thou didst deny the gold's receipt,
	And told'st me of a mistress and a dinner;
	For which, I hope, thou felt'st I was displeas'd. I pray you, master, tell me. Hold, take thou that, and that. Because that I familiarly sometimes
	Do use you for my fool, and chat with you,
	Your sauciness will jest upon my love,
	And make a common of my serious hours. When the sun shines let foolish gnats make sport,
	But creep in crannies when he hides his beams. If you will jest with me, know my aspect,
	And fashion your demeanour to my looks,
	Or I will beat this method in your sconce. so you would leave battering, I had rather have it a head: an you use these blows long, I must get a sconce for my head and insconce it too; or else I shall seek my wit in my shoulders. Nothing, sir, but that I am beaten. Ay, sir, and wherefore; for they say every why hath a wherefore. Why, first,—for flouting me; and then, wherefore,—
	For urging it the second time to me. Well, sir, I thank you. Marry, sir, for this something that you gave me for nothing. I'll make you amends next, to give you nothing for something. No, sir: I think the meat wants that I have



	In good time, sir; what's that? Well, sir, then 'twill be dry. If it be, sir, I pray you eat none of it. Lest it make you choleric, and purchase me another dry basting. Well, sir, learn to jest in good time: there's a time for all things. I durst have denied that, before you were so choleric. Marry, sir, by a rule as plain as the plain bald pate of Father Time himself. Yes, to pay a fine for a periwig and recover the lost hair of another man. Because it is a blessing that he bestows on beasts: and what he hath scanted men in hair, he hath given them in wit. Why, but there's many a man hath more hair than wit. Why, thou didst conclude hairy men plain dealers without wit. The plainer dealer, the sooner lost: yet be loseth it in a kind of jollity. Nay, not sound, I pray you. Nay, not sure, in a thing falsing. Certain ones, then. The one, to save the money that he spends in tiring; the other, that at dinner they should not drop in his porridge. Marry, and did, sir; namely, no time to recover hair lost by nature. But your reason was not substantial, why there is not time to recover. Thus I mend it: Time himself is bald, and therefore to the world's end will have bald followers. Ay, ay, Antipholus, look strange, and frown:
	Some other mistress hath thy sweet aspects,
	I am not Adriana, nor thy wife. The time was once when thou unurg'd wouldst vow
	That never words were music to thine ear,
	That never object pleasing in thine eye,
	That never touch well welcome to thy hand,
	That never meat sweet-savour'd in thy taste,
	Unless I spake, or look'd, or touch'd, or carv'd to thee. Thyself I call it, being strange to me,
	That, undividable, incorporate,
	Am better than thy dear self's better part. do not tear away thyself from me,
	For know, my love, as easy mayst thou fall
	A drop of water in the breaking gulf,
	And take unmingled thence that drop again,
	Without addition or diminishing,
	As take from me thyself and not me too. Wouldst thou not spit at me and spurn at me,
	And hurl the name of husband in my face,
	And tear the stain'd skin off my harlot-brow,
	And from my false hand cut the wedding-ring
	And break it with a deep-divorcing vow? I know thou canst; and therefore, see thou do it. I am possess'd with an adulterate blot;
	My blood is mingled with the crime of lust:
	For if we two be one and thou play false,
	I do digest the poison of thy flesh,
	Being strumpeted by thy contagion. Keep then fair league and truce with thy true bed;
	I live unstain'd, thou undishonoured. I know you not:
	In Ephesus I am but two hours old,
	As strange unto your town as to your talk;
	Who, every word by all my wit being scann'd,
	Want wit in all one word to understand. By thee; and this thou didst return from him,
	That he did buffet thee, and in his blows,
	Denied my house for his, me for his wife. Villain, thou liest; for even her very words
	Didst thou deliver to me on the mart. Be it my wrong you are from me exempt,
	But wrong not that wrong with a more contempt. Come, I will fasten on this sleeve of thine;
	Thou art an elm, my husband, I a vine,
	Whose weakness, married to thy stronger state,
	Makes me with thy strength to communicate:
	If aught possess thee from me, it is dross,
	Usurping ivy, brier, or idle moss;
	Who, all for want of pruning, with intrusion
	Infect thy sap and live on thy confusion. Until I know this sure uncertainty,
	I'll entertain the offer'd fallacy. We talk with goblins, owls, and elvish sprites:
	If we obey them not, this will ensue,
	They'll suck our breath, or pinch us black and blue. I think thou art, in mind, and so am I.



	Nay, master, both in mind and in my shape. No, I am an ape. If thou art chang'd to aught, 'tis to an ass. 'Tis so, I am an ass; else it could never be
	But I should know her as well as she knows me. Come, come; no longer will I be a fool,
	To put the finger in the eye and weep,
	Whilst man and master laugh my woes to scorn. Come, sir, to dinner. Dromio, keep the gate. Husband, I'll dine above with you to-day,
	And shrive you of a thousand idle pranks. Sirrah, if any ask you for your master,
	Say he dines forth, and let no creature enter. Come, sister. Dromio, play the porter well. I'll say as they say, and persever so,
	And in this mist at all adventures go. Ay; and let none enter, lest I break your pate. Come, come, Antipholus; we dine too late. Good Signior Angelo, you must excuse us all;
	My wife is shrewish when I keep not hours;
	Say that I linger'd with you at your shop
	To see the making of her carkanet,
	And that to-morrow you will bring it home. But here's a villain, that would face me down
	He met me on the mart, and that I beat him,
	And charg'd him with a thousand marks in gold,
	And that I did deny my wife and house. Say what you will, sir, but I know what I know;
	That you beat me at the mart, I have your hand to show:
	If the skin were parchment and the blows you gave were ink,
	Your own handwriting would tell you what I think. Marry, so it doth appear
	By the wrongs I suffer and the blows I bear. I should kick, being kick'd; and, being at that pass,
	You would keep from my heels and beware of an ass. You are sad, Signior Balthazar: pray God, our cheer
	May answer my good will and your good welcome here. I hold your dainties cheap, sir, and your welcome dear. O, Signior Balthazar, either at flesh or fish,
	A table-full of welcome makes scarce one dainty dish. Good meat, sir, is common; that every churl affords. And welcome more common, for that's nothing but words. Ay, to a niggardly host and more sparing guest:
	But though my cates be mean, take them in good part;
	Better cheer may you have, but not with better heart. Dost thou conjure for wenches, that thou call'st for such store,
	When one is one too many? Go, get thee from the door. Let him walk from whence he came, lest he catch cold on's feet. Right, sir; I'll tell you when, an you'll tell me wherefore. The porter for this time, sir, and my name is Dromio. thou hast stolen both mine office and my name:
	The one ne'er got me credit, the other mickle blame. If thou hadst been Dromio to-day in my place,
	Thou wouldst have chang'd thy face for a name, or thy name for an ass. Let my master in, Luce. Faith, no; he comes too late;
	And so tell your master. If thy name be call'd Luce,—Luce, thou hast answer'd him well. And you said, no. Thou baggage, let me in. Master, knock the door hard. You'll cry for this, minion, if I beat the door down. go, get you from the door. If you went in pain, master, this 'knave' would go sore. Here is neither cheer, sir, nor welcome: we would fain have either. In debating which was best, we shall part with neither. They stand at the door, master: bid them welcome hither. There is something in the wind, that we cannot get in. You would say so, master, if your garments were thin. Break any breaking here, and I'll break your knave's pate. A man may break a word with you, sir, and words are but wind:
	Ay, and break it in your face, so he break it not behind. It seems thou wantest breaking: out upon thee, hind! I pray thee, let me in. Ay, when fowls have no feathers, and fish have no fin. Well, I'll break in. For a fish without a fin, there's a fowl without a feather:
	If a crow help us in, sirrah, we'll pluck a crow together. let it not be so;
	Herein you war against your reputation,
	And draw within the compass of suspect
	The unviolated honour of your wife. Once this,—your long experience of her wisdom,
	Her sober virtue, years, and modesty,
	Plead on her part some cause to you unknown;
	And doubt not, sir, but she will well excuse
	Why at this time the doors are made against you. Be rul'd by me: depart in patience,
	And let us to the Tiger all to dinner;
	And about evening come yourself alone,
	To know the reason of this strange restraint. If by strong hand you offer to break in
	Now in the stirring passage of the day,
	A vulgar comment will be made of it,
	And that supposed by the common rout
	Against your yet ungalled estimation,
	That may with foul intrusion enter in
	And dwell upon your grave when you are dead;
	For slander lives upon succession,
	For ever housed where it gets possession. You have prevail'd: I will depart in quiet,
	And, in despite of mirth, mean to be merry. I know a wench of excellent discourse,
	Pretty and witty, wild and yet, too, gentle:
	There will we dine: this woman that I mean,
	My wife,—but, I protest, without desert,—
	Hath oftentimes upbraided me withal:
	To her will we to dinner. Get you home,
	And fetch the chain; by this I know 'tis made:
	Bring it, I pray you, to the Porpentine;
	For there's the house: that chain will I bestow,
	Be it for nothing but to spite my wife,
	Upon mine hostess there. Good sir, make haste. Since mine own doors refuse to entertain me,
	I'll knock elsewhere, to see if they'll disdain me. If you did wed my sister for her wealth,
	Then, for her wealth's sake use her with more kindness:
	Or, if you like elsewhere, do it by stealth;
	Muffle your false love with some show of blindness;
	Let not my sister read it in your eye;
	Be not thy tongue thy own shame's orator;
	Look sweet, speak fair, become disloyalty;
	Apparel vice like virtue's harbinger;
	Bear a fair presence, though your heart be tainted;
	Teach sin the carriage of a holy saint;
	Be secret-false: what need she be acquainted? 'Tis double wrong to truant with your bed,
	And let her read it in thy looks at board:
	Shame hath a bastard fame, well managed;
	Ill deeds are doubled with an evil word. poor women, make us but believe,
	Being compact of credit, that you love us;
	Though others have the arm, show us the sleeve;
	We in your motion turn, and you may move us. Then, gentle brother, get you in again;
	Comfort my sister, cheer her, call her wife:
	'Tis holy sport to be a little vain,
	When the sweet breath of flattery conquers strife. Sweet mistress,—what your name is else, I know not,
	Nor by what wonder you do hit of mine,—
	Less in your knowledge and your grace you show not
	Than our earth's wonder; more than earth divine. Teach me, dear creature, how to think and speak:
	Lay open to my earthy-gross conceit,
	Smother'd in errors, feeble, shallow, weak,
	The folded meaning of your words' deceit. Transform me then, and to your power I'll yield. But if that I am I, then well I know
	Your weeping sister is no wife of mine,
	Nor to her bed no homage do I owe:
	Far more, far more, to you do I decline. train me not, sweet mermaid, with thy note,
	To drown me in thy sister flood of tears:
	Sing, siren, for thyself, and I will dote:
	Spread o'er the silver waves thy golden hairs,
	And as a bed I'll take them and there lie;
	And, in that glorious supposition think
	He gains by death that hath such means to die:
	Let Love, being light, be drowned if she sink! Not mad, but mated; how, I do not know. For gazing on your beams; fair sun, being by. Gaze where you should, and that will clear your sight. As good to wink, sweet love, as look on night. No;
	It is thyself, mine own self's better part;
	Mine eye's clear eye, my dear heart's dearer heart;
	My food, my fortune, and my sweet hope's aim,
	My sole earth's heaven, and my heaven's claim. All this my sister is, or else should be. Call thyself sister, sweet, for I aim thee. soft, sir; hold you still:
	I'll fetch my sister, to get her good will. Thou art Dromio, thou art my man, thou art thyself. I am an ass, I am a woman's man and besides myself. Marry, sir, besides myself, I am due to a woman; one that claims me, one that haunts me, one that will have me. Marry, sir, such claim as you would lay to your horse; and she would have me as a beast: not that, I being a beast, she would have me; but that she, being a very beastly creature, lays claim to me. A very reverent body; aye, such a one as a man may not speak of, without he say, 'Sir-reverence.' I have but lean luck in the match, and yet is she a wondrous fat marriage. Marry, sir, she's the kitchen-wench, and all grease; and I know not what use to put her to but to make a lamp of her and run from her by her own light. I warrant her rags and the tallow in them will burn a Poland winter; if she lives till doomsday, she'll burn a week longer than the whole world. Swart, like my shoe, but her face nothing like so clean kept: for why she sweats; a man may go over shoes in the grime of it. No, sir, 'tis in grain; Noah's flood could not do it. Nell, sir; but her name and three quarters,—that is, an ell and three quarters,—will not measure her from hip to hip. No longer from head to foot than from hip to hip: she is spherical, like a globe; I could find out countries in her. Marry, sir, in her buttocks: I found it out by the bogs. In her forehead; armed and reverted, making war against her heir. I looked for the chalky cliffs, but I could find no whiteness in them: but I guess it stood in her chin, by the salt rheum that ran between France and it. Faith, I saw not; but I felt it hot in her breath. O, sir! upon her nose, all o'er embellished with rubies, carbuncles, sapphires, declining their rich aspect to the hot breath of Spain, who sent whole armadoes of caracks to be ballast at her nose. O, sir! To conclude, this drudge, or diviner, laid claim to me; call'd me Dromio; swore I was assured to her; told me what privy marks I had about me, as the mark of my shoulder, the mole in my neck, the great wart on my left arm, that I, amazed, ran from her as a witch. And, I think, if my breast had not been made of faith and my heart of steel,
	She had transform'd me to a curtal dog and made me turn i' the wheel. Go hie thee presently post to the road:
	An if the wind blow any way from shore,
	I will not harbour in this town to-night:
	If any bark put forth, come to the mart,
	Where I will walk till thou return to me. If every one knows us and we know none,
	'Tis time, I think, to trudge, pack, and be gone. As from a bear a man would run for life,
	So fly I from her that would be my wife. There's none but witches do inhabit here,
	And therefore 'tis high time that I were hence. She that doth call me husband, even my soul
	Doth for a wife abhor; but her fair sister,
	Possess'd with such a gentle sovereign grace,
	Of such enchanting presence and discourse,
	Hath almost made me traitor to myself:
	But, lest myself be guilty to self-wrong,
	I'll stop mine ears against the mermaid's song. Ay, that's my name. I know it well, sir: lo, here is the chain. What please yourself, sir: I have made it for you. I bespoke it not



	Not once, nor twice, but twenty times you have. Go home with it and please your wife withal;
	And soon at supper-time I'll visit you,
	And then receive my money for the chain. I pray you, sir, receive the money now,
	For fear you ne'er see chain nor money more. You are a merry man, sir: fare you well. What I should think of this, I cannot tell:
	But this I think, there's no man is so vain
	That would refuse so fair an offer'd chain. I see, a man here needs not live by shifts,
	When in the streets he meets such golden gifts. I'll to the mart, and there for Dromio stay:
	If any ship put out, then straight away. You know since Pentecost the sum is due,
	And since I have not much importun'd you;
	Nor now I had not, but that I am bound
	To Persia, and want guilders for my voyage:
	Therefore make present satisfaction,
	Or I'll attach you by this officer. Pleaseth you walk with me down to his house,
	I will discharge my bond, and thank you too. While I go to the goldsmith's house, go thou
	And buy a rope's end, that I will bestow
	Among my wife and her confederates,
	For locking me out of my doors by day. Get thee gone;
	Buy thou a rope, and bring it home to me. Belike you thought our love would last too long,
	If it were chain'd together, and therefore came not. Saving your merry humour, here's the note
	How much your chain weighs to the utmost carat. The fineness of the gold, and chargeful fashion,
	Which doth amount to three odd ducats more
	Than I stand debted to this gentleman:
	I pray you see him presently discharg'd,
	For he is bound to sea and stays but for it. I am not furnish'd with the present money;
	Besides, I have some business in the town. Good signior, take the stranger to my house,
	And with you take the chain, and bid my wife
	Disburse the sum on the receipt thereof:
	Perchance I will be there as soon as you. No; bear it with you, lest I come not time enough. Well, sir, I will. An if I have not, sir, I hope you have,
	Or else you may return without your money. Nay, come, I pray you, sir, give me the chain:
	Both wind and tide stays for this gentleman,
	And I, to blame, have held him here too long. I should have child you for not bringing it,
	But, like a shrew, you first begin to brawl. The hour steals on; I pray you, sir, dispatch. Why, give it to my wife and fetch your money. Come, come; you know I gave it you even now. I pray you, let me see it. Good sir, say whe'r you'll answer me or no:
	If not, I'll leave him to the officer. You wrong me more, sir, in denying it:
	Consider how it stands upon my credit. Well, officer, arrest him at my suit. Either consent to pay this sum for me,
	Or I attach you by this officer. Arrest me, foolish fellow, if thou dar'st. Here is thy fee: arrest him, officer. I would not spare my brother in this case,
	If he should scorn me so apparently. I do arrest you, sir: you hear the suit. But, sirrah, you shall buy this sport as dear
	As all the metal in your shop will answer. Sir, sir, I shall have law in Ephesus,
	To your notorious shame, I doubt it not. Master, there is a bark of Epidamnum
	That stays but till her owner comes aboard,
	And then she bears away. Our fraughtage, sir,
	I have convey'd aboard, and I have bought
	The oil, the balsamum, and aqua-vitæ. The ship is in her trim; the merry wind
	Blows fair from land; they stay for nought at all
	But for their owner, master, and yourself. A ship you sent me to, to hire waftage. Thou drunken slave, I sent thee for a rope;
	And told thee to what purpose, and what end. You sent me for a rope's end as soon:
	You sent me to the bay, sir, for a bark. I will debate this matter at more leisure,
	And teach your ears to list me with more heed. To Adriana, villain, hie thee straight;
	Give her this key, and tell her, in the desk
	That's cover'd o'er with Turkish tapestry,
	There is a purse of ducats: let her send it. Tell her I am arrested in the street,
	And that shall bail me. On, officer, to prison till it come. that is where we din'd,
	Where Dowsabel did claim me for her husband:
	She is too big, I hope, for me to compass. Thither I must, although against my will,
	For servants must their masters' minds fulfil. And true he swore, though yet forsworn he were. First, he did praise my beauty, then my speech. Have patience, I beseech. I cannot, nor I will not hold me still:
	My tongue, though not my heart, shall have his will. He is deformed, crooked, old and sere,
	Ill-fac'd, worse bodied, shapeless every where:
	Vicious, ungentle, foolish, blunt, unkind,
	Stigmatical in making, worse in mind. but I think him better than I say,
	And yet would herein others' eyes were worse. Far from her nest the lapwing cries away:
	My heart prays for him, though my tongue do curse. sweet, now, make haste. No, he's in Tartar limbo, worse than hell. A devil in an everlasting garment hath him,
	One whose hard heart is button'd up with steel;
	A fiend, a fairy, pitiless and rough;
	A wolf, nay, worse, a fellow all in buff;
	A back-friend, a shoulder-clapper, one that countermands
	The passages of alleys, creeks and narrow lands;
	A hound that runs counter and yet draws dryfoot well;
	One that, before the judgment, carries poor souls to hell. I know not at whose suit he is arrested well;
	But he's in a suit of buff which 'rested him, that can I tell. Go fetch it, sister.—

 This I wonder at:
	That he, unknown to me, should be in debt:
	Tell me, was he arrested on a band? Not on a band, but on a stronger thing;
	A chain, a chain. No, no, the bell: 'tis time that I were gone:
	It was two ere I left him, and now the clock strikes one. O yes; if any hour meet a sergeant, a' turns back for very fear. Time is a very bankrupt, and owes more than he's worth to season. Nay, he's a thief too: have you not heard men say,
	That Time comes stealing on by night and day? Go, Dromio: there's the money, bear it straight,
	And bring thy master home immediately. Come, sister; I am press'd down with conceit; Conceit, my comfort and my injury. There's not a man I meet but doth salute me,
	As if I were their well acquainted friend;
	And every one doth call me by my name. Some tender money to me; some invite me;
	Some other give me thanks for kindnesses;
	Some offer me commodities to buy:
	Even now a tailor call'd me in his shop
	And show'd me silks that he had bought for me,
	And therewithal, took measure of my body. Sure these are but imaginary wiles,
	And Lapland sorcerers inhabit here. Master, here's the gold you sent me for. Not that Adam that kept the Paradise, but that Adam that keeps the prison: he that goes in the calf's skin that was killed for the Prodigal: he that came behind you, sir, like an evil angel, and bid you forsake your liberty. why, 'tis a plain case: he that went, like a base-viol, in a case of leather; the man, sir, that, when gentlemen are tired, gives them a fob, and 'rests them; he, sir, that takes pity on decayed men and gives them suits of durance; he that sets up his rest to do more exploits with his mace than a morris-pike. Ay, sir, the sergeant of the band; he that brings any man to answer it that breaks his band; one that thinks a man always going to bed, and says, 'God give you good rest!' Well, sir, there rest in your foolery. Why, sir, I brought you word an hour since that the bark Expedition put forth to-night; and then were you hindered by the sergeant to tarry for the hoy Delay. The fellow is distract, and so am I;
	And here we wander in illusions:
	Some blessed power deliver us from hence! Well met, well met, Master Antipholus. I see, sir, you have found the goldsmith now:
	Is that the chain you promis'd me to-day? Nay, she is worse, she is the devil's dam, and here she comes in the habit of a light wench: and thereof comes that the wenches say, 'God damn me;' that's as much as to say, 'God make me a light wench.' It is written, they appear to men like angels of light: light is an effect of fire, and fire will burn; ergo, light wenches will burn. Your man and you are marvellous merry, sir. Master, if you do, expect spoon-meat, so bespeak a long spoon. Marry, he must have a long spoon that must eat with the devil. Thou art, as you are all, a sorceress:
	I conjure thee to leave me and be gone. Give me the ring of mine you had at dinner,
	Or, for my diamond, the chain you promis'd,
	And I'll be gone, sir, and not trouble you. Some devils ask but the parings of one's nail,
	A rush, a hair, a drop of blood, a pin,
	A nut, a cherry-stone;
	But she, more covetous, would have a chain. Master, be wise: an if you give it her,
	The devil will shake her chain and fright us with it. I pray you, sir, my ring, or else the chain:
	I hope you do not mean to cheat me so. Come, Dromio, let us go. 'Fly pride,' says the peacock: mistress, that you know. Now, out of doubt, Antipholus is mad,
	Else would he never so demean himself. A ring he hath of mine worth forty ducats,
	And for the same he promis'd me a chain:
	Both one and other he denies me now. The reason that I gather he is mad,
	Besides this present instance of his rage,
	Is a mad tale he told to-day at dinner,
	Of his own doors being shut against his entrance
	Belike his wife, acquainted with his fits,
	On purpose shut the doors against his way. My way is now to hie home to his house,
	And tell his wife, that, being lunatic,
	He rush'd into my house, and took perforce
	My ring away. This course I fittest choose,
	For forty ducats is too much to lose. Fear me not, man; I will not break away:
	I'll give thee, ere I leave thee, so much money,
	To warrant thee, as I am 'rested for. My wife is in a wayward mood to-day,
	And will not lightly trust the messenger. That I should be attach'd in Ephesus,
	I tell you, 'twill sound harshly in her ears. How now, sir! Here's that, I warrant you, will pay them all. Why, sir, I gave the money for the rope. I'll serve you, sir, five hundred at the rate. To a rope's end, sir; and to that end am I return'd. And to that end, sir, I will welcome you. Good sir, be patient. Nay, 'tis for me to be patient; I am in adversity. Good now, hold thy tongue. Nay, rather persuade him to hold his hands. I would I were senseless, sir, that I might not feel your blows. Thou art sensible in nothing but blows, and so is an ass. I have served him from the hour of my nativity to this instant, and have nothing at his hands for my service but blows. When I am cold, he heats me with beating; when I am warm, he cools me with beating; I am waked with it when I sleep; raised with it when I sit; driven out of doors with it when I go from home; welcomed home with it when I return; nay, I bear it on my shoulders, as a beggar wont her brat; and, I think, when he hath lamed me, I shall beg with it from door to door. Come, go along; my wife is coming yonder. Mistress, respice finem, respect your end; or rather, to prophesy like the parrot, 'Beware the rope's end.' Good Doctor Pinch, you are a conjurer;
	Establish him in his true sense again,
	And I will please you what you will demand. There is my hand, and let it feel your ear. I charge thee, Satan, hous'd within this man,
	To yield possession to my holy prayers,
	And to thy state of darkness hie thee straight:
	I conjure thee by all the saints in heaven. O husband, God doth know you din'd at home;
	Where would you had remain'd until this time. Sir, sooth to say, you did not dine at home. Perdy, your doors were lock'd and you shut out. Sans fable, she herself revil'd you there. Certes, she did; the kitchen-vestal scorn'd you. In verity you did: my bones bear witness,
	That since have felt the vigour of his rage. It is no shame: the fellow finds his vein,
	And, yielding to him humours well his frenzy. I sent you money to redeem you,
	By Dromio here, who came in haste for it. heart and good will you might;
	But surely, master, not a rag of money. He came to me, and I deliver'd it. Mistress, both man and master is possess'd:
	I know it by their pale and deadly looks. I did not, gentle husband, lock thee forth. And, gentle master, I receiv'd no gold;
	But I confess, sir, that we were lock'd out. bind him, bind him, let him not come near me. Thou gaoler, thou,
	I am thy prisoner: wilt thou suffer them
	To make a rescue? Masters, let him go:
	He is my prisoner, and you shall not have him. Go bind this man, for he is frantic too. He is my prisoner: if I let him go,
	The debt he owes will be requir'd of me. I will discharge thee ere I go from thee:
	Bear me forthwith unto his creditor,
	And, knowing how the debt grows, I will pay it. Good Master doctor, see him safe convey'd
	Home to my house. Master, I am here enter'd in bond for you. be mad, good master; cry, 'the devil!' Sister, go you with me.—



	Say now, whose suit is he arrested at? One Angelo, a goldsmith; do you know him? He did bespeak a chain for me, but had it not. When as your husband all in rage, to-day
	Came to my house, and took away my ring,—
	The ring I saw upon his finger now,—
	Straight after did I meet him with a chain. It may be so, but I did never see it. Come, gaoler, bring me where the goldsmith is:
	I long to know the truth hereof at large. I see, these witches are afraid of swords. Faith, stay here this night, they will surely do us no harm; you saw they speak us fair, give us gold: methinks they are such a gentle nation, that, but for the mountain of mad flesh that claims marriage of me, I could find in my heart to stay here still, and turn witch. I will not stay to-night for all the town;
	Therefore away, to get our stuff aboard. I am sorry, sir, that I have hinder'd you;
	But, I protest, he had the chain of me,
	Though most dishonestly he doth deny it. Of very reverend reputation, sir,
	Of credit infinite, highly belov'd,
	Second to none that lives here in the city:
	His word might bear my wealth at any time. Speak softly: yonder, as I think, he walks. Good sir, draw near to me, I'll speak to him. Signior Antipholus, I wonder much
	That you would put me to this shame and trouble;
	And not without some scandal to yourself,
	With circumstance and oaths so to deny
	This chain which now you wear so openly:
	Beside the charge, the shame, imprisonment,
	You have done wrong to this my honest friend,
	Who, but for staying on our controversy,
	Had hoisted sail and put to sea to-day. Yes, that you did, sir, and forswore it too. These ears of mine, thou know'st, did hear thee. Thou art a villain to impeach me thus:
	I'll prove mine honour and mine honesty
	Against thee presently, if thou dar'st stand. I dare, and do defy thee for a villain. Some get within him, take his sword away. Bind Dromio too, and bear them to my house. Run, master, run; for God's sake, take a house! This is some priory: in, or we are spoil'd. Be quiet, people. Let us come in, that we may bind him fast,
	And bear him home for his recovery. This week he hath been heavy, sour, sad,
	And much different from the man he was;
	But, till this afternoon his passion
	Ne'er brake into extremity of rage. A sin prevailing much in youthful men,
	Who give their eyes the liberty of gazing. To none of these, except it be the last;
	Namely, some love that drew him oft from home. Why, so I did. Ay, but not rough enough. Haply, in private. Ay, but not enough. It was the copy of our conference:
	In bed, he slept not for my urging it;
	At board, he fed not for my urging it;
	Alone, it was the subject of my theme;
	In company I often glanced it:
	Still did I tell him it was vile and bad. It seems, his sleeps were hinder'd by thy railing,
	And thereof comes it that his head is light. Thou say'st his sports were hinder'd by thy brawls:
	Sweet recreation barr'd, what doth ensue
	But moody moping, and dull melancholy,
	Kinsman to grim and comfortless despair,
	And at her heels a huge infectious troop
	Of pale distemperatures and foes to life? In food, in sport, and life-preserving rest
	To be disturb'd, would mad or man or beast:
	The consequence is then, thy jealous fits
	Have scar'd thy husband from the use of wits. She never reprehended him but mildly
	When he demean'd himself rough, rude, and wildly. Good people, enter, and lay hold on him. Then, let your servants bring my husband forth. Neither: he took this place for sanctuary,
	And it shall privilege him from your hands
	Till I have brought him to his wits again,
	Or lose my labour in assaying it. I will attend my husband, be his nurse,
	Diet his sickness, for it is my office,
	And will have no attorney but myself;
	And therefore let me have him home with me. Be patient; for I will not let him stir
	Till I have us'd the approved means I have,
	With wholesome syrups, drugs, and holy prayers,
	To make of him a formal man again. It is a branch and parcel of mine oath,
	A charitable duty of my order;
	Therefore depart and leave him here with me. Be quiet, and depart: thou shalt not have him. Come, go: I will fall prostrate at his feet,
	And never rise until my tears and prayers
	Have won his Grace to come in person hither,
	And take perforce my husband from the abbess. By this, I think, the dial points at five:
	Anon, I'm sure, the duke himself in person
	Comes this way to the melancholy vale,
	The place of death and sorry execution,
	Behind the ditches of the abbey here. To see a reverend Syracusian merchant,
	Who put unluckily into this bay
	Against the laws and statutes of this town,
	Beheaded publicly for his offence. Yet once again proclaim it publicly,
	If any friend will pay the sum for him,
	He shall not die; so much we tender him. May it please your Grace, Antipholus, my husband,
	Whom I made lord of me and all I had,
	At your important letters, this ill day
	A most outrageous fit of madness took him,
	That desperately he hurried through the street,—
	With him his bondman, all as mad as he,—
	Doing displeasure to the citizens
	By rushing in their houses, bearing thence
	Rings, jewels, anything his rage did like. Once did I get him bound and sent him home,
	Whilst to take order for the wrongs I went
	That here and there his fury had committed. Anon, I wot not by what strong escape,
	He broke from those that had the guard of him,
	And with his mad attendant and himself,
	Each one with ireful passion, with drawn swords
	Met us again, and, madly bent on us
	Chas'd us away, till, raising of more aid
	We came again to bind them. Then they fled
	Into this abbey, whither we pursu'd them;
	And here the abbess shuts the gates on us,
	And will not suffer us to fetch him out,
	Nor send him forth that we may bear him hence. Therefore, most gracious duke, with thy command
	Let him be brought forth, and borne hence for help. Long since thy husband serv'd me in my wars,
	And I to thee engag'd a prince's word,
	When thou didst make him master of thy bed,
	To do him all the grace and good I could. Go, some of you, knock at the abbey gate
	And bid the lady abbess come to me. My master and his man are both broke loose,
	Beaten the maids a-row and bound the doctor,
	Whose beard they have sing'd off with brands of fire;
	And ever as it blaz'd they threw on him
	Great pails of puddled mire to quench the hair. My master preaches patience to him, and the while
	His man with scissors nicks him like a fool;
	And sure, unless you send some present help,
	Between them they will kill the conjurer. thy master and his man are here,
	And that is false thou dost report to us. Mistress, upon my life, I tell you true;
	I have not breath'd almost, since I did see it. He cries for you and vows, if he can take you,
	To scotch your face, and to disfigure you. Come, stand by me; fear nothing. Witness you,
	That he is borne about invisible:
	Even now we hous'd him in the abbey here,
	And now he's here, past thought of human reason. grant me justice,
	Even for the service that long since I did thee,
	When I bestrid thee in the wars and took
	Deep scars to save thy life; even for the blood
	That then I lost for thee, now grant me justice. She whom thou gav'st to me to be my wife,
	That hath abused and dishonour'd me,
	Even in the strength and height of injury! Discover how, and thou shalt find me just. This day, great duke, she shut the doors upon me,
	While she with harlots feasted in my house. No, my good lord: myself, he, and my sister
	To-day did dine together. My liege, I am advised what I say:
	Neither disturb'd with the effect of wine,
	Nor heady-rash, provok'd with raging ire,
	Albeit my wrongs might make one wiser mad. This woman lock'd me out this day from dinner:
	That goldsmith there, were he not pack'd with her,
	Could witness it, for he was with me then;
	Who parted with me to go fetch a chain,
	Promising to bring it to the Porpentine,
	Where Balthazar and I did dine together. Our dinner done, and he not coming thither,
	I went to seek him: in the street I met him,
	And in his company that gentleman. There did this perjur'd goldsmith swear me down
	That I this day of him receiv'd the chain,
	Which, God he knows, I saw not; for the which
	He did arrest me with an officer. I did obey, and sent my peasant home
	For certain ducats: he with none return'd. By the way we met
	My wife, her sister, and a rabble more
	Of vile confederates: along with them
	They brought one Pinch, a hungry lean-fac'd villain,
	A mere anatomy, a mountebank,
	A threadbare juggler, and a fortune-teller,
	A needy, hollow-ey'd, sharp-looking wretch,
	A living-dead man. This pernicious slave,
	Forsooth, took on him as a conjurer,
	And, gazing in mine eyes, feeling my pulse,
	And with no face, as 'twere, out-facing me,
	Cries out, I was possess'd. Then, altogether
	They fell upon me, bound me, bore me thence,
	And in a dark and dankish vault at home
	There left me and my man, both bound together;
	Till, gnawing with my teeth my bonds in sunder,
	I gain'd my freedom, and immediately
	Ran hither to your Grace; whom I beseech
	To give me ample satisfaction
	For these deep shames and great indignities. My lord, in truth, thus far I witness with him,
	That he din'd not at home, but was lock'd out. He had, my lord; and when he ran in here,
	These people saw the chain about his neck. Besides, I will be sworn these ears of mine
	Heard you confess you had the chain of him
	After you first forswore it on the mart;
	And thereupon I drew my sword on you;
	And then you fled into this abbey here,
	From whence, I think, you are come by miracle. I never came within these abbey walls;
	Nor ever didst thou draw thy sword on me;
	I never saw the chain, so help me heaven! If here you hous'd him, here he would have been;
	If he were mad, he would not plead so coldly;
	You say he din'd at home; the goldsmith here
	Denies that saying. Sir, he din'd with her there, at the Porpentine. He did, and from my finger snatch'd that ring. 'Tis true, my liege; this ring I had of her. As sure, my liege, as I do see your Grace. Why, this is strange. Most mighty duke, vouchsafe me speak a word:
	Haply I see a friend will save my life,
	And pay the sum that may deliver me. Speak freely, Syracusian, what thou wilt. Within this hour I was his bondman, sir;
	But he, I thank him, gnaw'd in two my cords:
	Now am I Dromio and his man, unbound. Ourselves we do remember, sir, by you;
	For lately we were bound, as you are now. grief hath chang'd me since you saw me last,
	And careful hours, with Time's deformed hand,
	Have written strange defeatures in my face:
	But tell me yet, dost thou not know my voice? No, trust me, sir, not I.



	I am sure thou dost. Ay, sir; but I am sure I do not; and whatsoever a man denies, you are now bound to believe him. O, time's extremity,
	Hast thou so crack'd and splitted my poor tongue
	In seven short years, that here my only son
	Knows not my feeble key of untun'd cares? Though now this grained face of mine be hid
	In sap-consuming winter's drizzled snow,
	And all the conduits of my blood froze up,
	Yet hath my night of life some memory,
	My wasting lamps some fading glimmer left,
	My dull deaf ears a little use to hear:
	All these old witnesses, I cannot err,
	Tell me thou art my son Antipholus. But seven years since, in Syracusa, boy,
	Thou know'st we parted: but perhaps, my son,
	Thou sham'st to acknowledge me in misery. I tell thee, Syracusian, twenty years
	Have I been patron to Antipholus,
	During which time he ne'er saw Syracusa. Most mighty duke, behold a man much wrong'd. One of these men is Genius to the other;
	And so of these: which is the natural man,
	And which the spirit? I, sir, am Dromio: command him away. I, sir, am Dromio: pray let me stay. Whoever bound him, I will loose his bonds,
	And gain a husband by his liberty. Speak, old Ægeon, if thou be'st the man
	That hadst a wife once call'd Æmilia,
	That bore thee at a burden two fair sons. If I dream not, thou art Æmilia:
	If thou art she, tell me where is that son
	That floated with thee on the fatal raft? By men of Epidamnum, he and I,
	And the twin Dromio, all were taken up:
	But by and by rude fishermen of Corinth
	By force took Dromio and my son from them,
	And me they left with those of Epidamnum. What then became of them, I cannot tell;
	I to this fortune that you see me in. Why, here begins his morning story right:
	These two Antipholus', these two so like,
	And these two Dromios, one in semblance,
	Besides her urging of her wrack at sea;
	These are the parents to these children,
	Which accidentally are met together. No, sir, not I; I came from Syracuse. Stay, stand apart; I know not which is which. I came from Corinth, my most gracious lord,—



	And I with him. Brought to this town by that most famous warrior,
	Duke Menaphon, your most renowned uncle. I, gentle mistress. And so do I; yet did she call me so;
	And this fair gentlewoman, her sister here,
	Did call me brother. What I told you then,
	I hope I shall have leisure to make good,
	If this be not a dream I see and hear. That is the chain, sir, which you had of me. I think it be, sir; I deny it not. And you, sir, for this chain arrested me. I think I did, sir; I deny it not. I sent you money, sir, to be your bail,
	By Dromio; but I think he brought it not. No, none by me. This purse of ducats I receiv'd from you,
	And Dromio, my man, did bring them me. I see we still did meet each other's man,
	And I was ta'en for him, and he for me,
	And thereupon these errors are arose. Sir, I must have that diamond from you. There, take it; and much thanks for my good cheer. Renowned duke, vouchsafe to take the pains
	To go with us into the abbey here,
	And hear at large discoursed all our fortunes;
	And all that are assembled in this place,
	That by this sympathized one day's error
	Have suffer'd wrong, go keep us company,
	And we shall make full satisfaction. Thirty-three years have I but gone in travail
	Of you, my sons; and, till this present hour
	My heavy burdens ne'er delivered. The duke, my husband, and my children both,
	And you the calendars of their nativity,
	Go to a gossip's feast, and joy with me:
	After so long grief such festivity! Dromio, what stuff of mine hast thou embark'd? Your goods that lay at host, sir, in the Centaur. I am your master, Dromio:
	Come, go with us; we'll look to that anon:
	Embrace thy brother there; rejoice with him. There is a fat friend at your master's house,
	That kitchen'd me for you to-day at dinner:
	She now shall be my sister, not my wife. Methinks you are my glass, and not my brother:
	I see by you I am a sweet-fac'd youth. Not I, sir; you are my elder. Nay, then, thus:
	We came into the world like brother and brother;
	And now let's go hand in hand, not one before another.


"""
meteor_scores, bert_scores, rouge_scores = evaluate_summaries(generated_summary, reference_summary)

# Print evaluation scores
print("METEOR scores:")
for method, score in meteor_scores.items():
    print(f"{method}: {score}")
print()

print("BERT scores:")
for method, score in bert_scores.items():
    print(f"{method}: {score}")
print()

print("ROUGE scores:")
for method, score in rouge_scores.items():
    print(f"{method}: {score}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


METEOR scores:
Generated: 0.14132272025900494

BERT scores:
Generated: 0.7870688438415527

ROUGE scores:
Generated: {'rouge-1': {'r': 0.5239852398523985, 'p': 0.04588045234248788, 'f': 0.08437314171609318}, 'rouge-2': {'r': 0.13008130081300814, 'p': 0.007070260715863898, 'f': 0.013411566499084211}, 'rouge-l': {'r': 0.5055350553505535, 'p': 0.04426494345718902, 'f': 0.08140225639226668}}


In [ ]:
#9
#TF-IDF
generated_summary="""Dromio, thou drone, thou snail, thou slug, thou sot! Thou art Dromio, thou art my man, thou art thyself. Thou gaoler, thou,
	I am thy prisoner: wilt thou suffer them
	To make a rescue? Try all the friends thou hast in Ephesus;
	Beg thou, or borrow, to make up the sum,
	And live; if no, then thou art doom'd to die. If thou hadst been Dromio to-day in my place,
	Thou wouldst have chang'd thy face for a name, or thy name for an ass."""
meteor_scores, bert_scores, rouge_scores = evaluate_summaries(generated_summary, reference_summary)

# Print evaluation scores
print("METEOR scores:")
for method, score in meteor_scores.items():
    print(f"{method}: {score}")
print()

print("BERT scores:")
for method, score in bert_scores.items():
    print(f"{method}: {score}")
print()

print("ROUGE scores:")
for method, score in rouge_scores.items():
    print(f"{method}: {score}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


METEOR scores:
Generated: 0.0489853044086774

BERT scores:
Generated: 0.775224506855011

ROUGE scores:
Generated: {'rouge-1': {'r': 0.04797047970479705, 'p': 0.22033898305084745, 'f': 0.07878787585142345}, 'rouge-2': {'r': 0.0040650406504065045, 'p': 0.024691358024691357, 'f': 0.006980800364756927}, 'rouge-l': {'r': 0.04797047970479705, 'p': 0.22033898305084745, 'f': 0.07878787585142345}}


In [ ]:
#10
#sentence-ranking
generated_summary="""my youngest boy  and yet my eldest care   at eighteen years became inquisitive  after his brother  and importun d me  that his attendant for his case was like   reft of his brother  but retain d his name   might bear him company in the quest of him   whom whilst i labour d of a love to see   i hazarded the loss of whom i lov d
renowned duke  vouchsafe to take the pains  to go with us into the abbey here   and hear at large discoursed all our fortunes   and all that are assembled in this place   that by this sympathized one day s error  have suffer d wrong  go keep us company   and we shall make full satisfaction
come  i will fasten on this sleeve of thine   thou art an elm  my husband  i a vine   whose weakness  married to thy stronger state   makes me with thy strength to communicate   if aught possess thee from me  it is dross   usurping ivy  brier  or idle moss   who  all for want of pruning  with intrusion  infect thy sap and live on thy confusion
for  ere the ships could meet by twice five leagues   we were encounter d by a mighty rock   which being violently borne upon   our helpful ship was splitted in the midst   so that  in this unjust divorce of us  fortune had left to both of us alike  what to delight in  what to sorrow for
may it please your grace  antipholus  my husband   whom i made lord of me and all i had   at your important letters  this ill day  a most outrageous fit of madness took him   that desperately he hurried through the street    with him his bondman  all as mad as he    doing displeasure to the citizens  by rushing in their houses  bearing thence  rings  jewels  anything his rage did like """
meteor_scores, bert_scores, rouge_scores = evaluate_summaries(generated_summary, reference_summary)

# Print evaluation scores
print("METEOR scores:")
for method, score in meteor_scores.items():
    print(f"{method}: {score}")
print()

print("BERT scores:")
for method, score in bert_scores.items():
    print(f"{method}: {score}")
print()

print("ROUGE scores:")
for method, score in rouge_scores.items():
    print(f"{method}: {score}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


METEOR scores:
Generated: 0.11588563694806293

BERT scores:
Generated: 0.7668247222900391

ROUGE scores:
Generated: {'rouge-1': {'r': 0.14022140221402213, 'p': 0.19689119170984457, 'f': 0.16379309858956984}, 'rouge-2': {'r': 0.01016260162601626, 'p': 0.01694915254237288, 'f': 0.012706475618251913}, 'rouge-l': {'r': 0.12546125461254612, 'p': 0.17616580310880828, 'f': 0.14655171927922503}}


In [ ]:
#11
#TextRank
generated_summary="""If you did wed my sister for her wealth,
	Then, for her wealth's sake use her with more kindness:
	Or, if you like elsewhere, do it by stealth;
	Muffle your false love with some show of blindness;
	Let not my sister read it in your eye;
	Be not thy tongue thy own shame's orator;
	Look sweet, speak fair, become disloyalty;
	Apparel vice like virtue's harbinger;
	Bear a fair presence, though your heart be tainted;
	Teach sin the carriage of a holy saint;
	Be secret-false: what need she be acquainted?
Faith, stay here this night, they will surely do us no harm; you saw they speak us fair, give us gold: methinks they are such a gentle nation, that, but for the mountain of mad flesh that claims marriage of me, I could find in my heart to stay here still, and turn witch.
A league from Epidamnum had we sail'd,
	Before the always-wind-obeying deep
	Gave any tragic instance of our harm:
	But longer did we not retain much hope;
	For what obscured light the heavens did grant
	Did but convey unto our fearful minds
	A doubtful warrant of immediate death;
	Which, though myself would gladly have embrac'd,
	Yet the incessant weepings of my wife,
	Weeping before for what she saw must come,
	And piteous plainings of the pretty babes,
	That mourn'd for fashion, ignorant what to fear,
	Forc'd me to seek delays for them and me.
She that doth call me husband, even my soul
	Doth for a wife abhor; but her fair sister,
	Possess'd with such a gentle sovereign grace,
	Of such enchanting presence and discourse,
	Hath almost made me traitor to myself:
	But, lest myself be guilty to self-wrong,
	I'll stop mine ears against the mermaid's song.
A wretched soul, bruis'd with adversity,
	We bid be quiet when we hear it cry;
	But were we burden'd with like weight of pain,
	As much, or more we should ourselves complain:
	So thou, that hast no unkind mate to grieve thee,
	With urging helpless patience wouldst relieve me:
	But if thou live to see like right bereft.

"""
meteor_scores, bert_scores, rouge_scores = evaluate_summaries(generated_summary, reference_summary)

# Print evaluation scores
print("METEOR scores:")
for method, score in meteor_scores.items():
    print(f"{method}: {score}")
print()

print("BERT scores:")
for method, score in bert_scores.items():
    print(f"{method}: {score}")
print()

print("ROUGE scores:")
for method, score in rouge_scores.items():
    print(f"{method}: {score}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


METEOR scores:
Generated: 0.14758604317961943

BERT scores:
Generated: 0.7742666006088257

ROUGE scores:
Generated: {'rouge-1': {'r': 0.12177121771217712, 'p': 0.1346938775510204, 'f': 0.1279069717568808}, 'rouge-2': {'r': 0.014227642276422764, 'p': 0.02046783625730994, 'f': 0.01678656590514778}, 'rouge-l': {'r': 0.11808118081180811, 'p': 0.1306122448979592, 'f': 0.12403100276463273}}


In [ ]:
#12
#PEGASUS
generated_summary="""In Syracusa was I born, and wed Unto a woman, happy but for me, And by me too, had not our hap been bad.<n>With her I liv'd in joy: our wealth increas'd By prosperous seafaring I often made To Epidamnum; till my factor's death, And the great care of goods at random left, Drew me from kind embracements of weeping wife."""
meteor_scores, bert_scores, rouge_scores = evaluate_summaries(generated_summary, reference_summary)

# Print evaluation scores
print("METEOR scores:")
for method, score in meteor_scores.items():
    print(f"{method}: {score}")
print()

print("BERT scores:")
for method, score in bert_scores.items():
    print(f"{method}: {score}")
print()

print("ROUGE scores:")
for method, score in rouge_scores.items():
    print(f"{method}: {score}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


METEOR scores:
Generated: 0.03430682617874736

BERT scores:
Generated: 0.7741801738739014

ROUGE scores:
Generated: {'rouge-1': {'r': 0.055350553505535055, 'p': 0.26785714285714285, 'f': 0.09174311642753613}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.05166051660516605, 'p': 0.25, 'f': 0.08562690847646579}}
